In [1]:
from utils import spark_session, save_table

session = spark_session("purchase-suppression-tmp")

24/11/07 12:37:02 INFO SecurityManager: Changing view acls to: olivyatan,b_perso,*
24/11/07 12:37:02 INFO SecurityManager: Changing modify acls to: olivyatan,b_perso
24/11/07 12:37:02 INFO SecurityManager: Changing view acls groups to: 
24/11/07 12:37:02 INFO SecurityManager: Changing modify acls groups to: 
24/11/07 12:37:02 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(olivyatan, b_perso, *); groups with view permissions: Set(); users  with modify permissions: Set(olivyatan, b_perso); groups with modify permissions: Set()
24/11/07 12:37:02 INFO deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
24/11/07 12:37:03 INFO SparkContext: Running Spark version 3.1.1.0.1.0
24/11/07 12:37:03 INFO ResourceUtils: ==============================================================
24/11/07 12:37:03 INFO ResourceUtils: No custom resources configured for spark.driver.
24/11/07 12:37:03 INFO ResourceUtil

## Load data:

In [2]:
import features as F

df_smp = session.table("bx_ps_samples_enriched_1")
df_prch = session.table("bx_ps_user_prch_hist_1")
df_vi = session.table("bx_ps_user_vi_hist_1")

In [3]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.sql import Window   

In [4]:
def add_feature(df_base, df_feature, feature_name=None, join_key="ITEM_ID", na_value=None, how="inner"):
    df_res = df_base.join(df_feature, on=["BUYER_ID", join_key, "EVENT_TIMESTAMP"], how=how)
   # if na_value:
    if na_value is not None and feature_name:
        df_res = df_res.fillna(na_value, subset=[feature_name])
    return df_res

In [5]:
def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 
    
def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME


## Features calc:

In [3]:
# time_since_last_action_from_cat_in_days
df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_prch, cat_type="LEAF", action_type="purchase")
df_res = F.add_feature(df_smp, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_prch, cat_type="LVL2", action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_prch, cat_type="META", action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_vi, cat_type="LEAF", action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_vi, cat_type="LVL2", action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_time_since_last_action_from_cat_in_days(df_smp, df_vi, cat_type="META", action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")


#num_actions_from_cat_in_last_x_days
df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=1, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=2, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=5, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=7, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=14, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=1, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=2, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=5, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=7, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=14, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=1, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=2, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=5, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=7, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=14, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_prch, cat_type="META", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=1, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=2, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=5, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=7, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=14, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=1, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=2, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=5, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=7, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=14, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=1, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=2, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=5, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=7, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=14, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_num_actions_from_cat_in_last_x_days(df_smp, df_vi, cat_type="META", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

In [4]:
#cat_propensity_sim
df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LEAF", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="LVL2", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_prch, action_type="purchase", cat_type="META", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LEAF", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="LVL2", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=7)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=14)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=30)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=60)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=90)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=180)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_cat_propensity_sim(df_smp, df_vi, action_type="view", cat_type="META", days=360)
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

In [ ]:
#title_propensity_sim
# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=7)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=14)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=30)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=60)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=90)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=180)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_prch, action_type="purchase", days=360)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=7)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=14)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=30)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=60)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=90)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=180)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

# df_ft = F.f_title_propensity_sim(df_smp, df_vi, action_type="view", days=360)
# df_res = F.add_feature(df_res, df_ft, join_key="ITEM_ID", how="left")

In [6]:
#save_table(session, df_res, "bx_ps_features_1")

In [3]:
# create a smaller version only with key, features and label
df = session.table("bx_ps_features_1")
f_columns = [col for col in df.columns if col.startswith('f_')]
f_columns.extend(["BUYER_ID", "LEAF_CATEG_ID", "EVENT_TIMESTAMP", "label", "num_repurchases"])
df = df.select(f_columns)

# split data to train/validation/test sets
train, validation, test = df.randomSplit([0.7, 0.1, 0.2])

# save
save_table(session, train, "bx_ps_features_train_1")
save_table(session, validation, "bx_ps_features_valid_1")
save_table(session, test, "bx_ps_features_test_1")

In [ ]:
if session:
    session.stop()

### Add features of user priors:

### Data validation:

In [3]:
df_res=session.table( "bx_ps_features_1")

In [12]:
# Check the number of columns
num_columns = len(df_res.columns)
print(f"Number of columns: {num_columns}")

# Check the number of rows
num_rows = df_res.count()
print(f"Number of rows: {num_rows}")

Number of columns: 832


Number of rows: 99999


In [4]:
sample_buyers = [row["BUYER_ID"] for row in df_res.select("BUYER_ID").distinct().limit(10).collect()]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_res_sample = df_res.filter(df_res.BUYER_ID.isin(sample_buyers))


In [5]:
df_res_sample.columns

['BUYER_ID',
 'META_CATEG_ID',
 'EVENT_TIMESTAMP',
 'CATEG_LVL2_ID',
 'LEAF_CATEG_ID',
 'num_repurchases',
 'label',
 'LEAF_CATEG_NAME',
 'CATEG_LVL2_NAME',
 'META_CATEG_NAME',
 'f_time_since_last_purchase_from_LEAF_cat',
 'f_time_since_last_purchase_from_LVL2_cat',
 'f_time_since_last_purchase_from_META_cat',
 'f_time_since_last_view_from_LEAF_cat',
 'f_time_since_last_view_from_LVL2_cat',
 'f_time_since_last_view_from_META_cat',
 'f_num_purchases_from_LEAF_cat_in_last_1_days',
 'f_num_purchases_from_LEAF_cat_in_last_2_days',
 'f_num_purchases_from_LEAF_cat_in_last_5_days',
 'f_num_purchases_from_LEAF_cat_in_last_7_days',
 'f_num_purchases_from_LEAF_cat_in_last_14_days',
 'f_num_purchases_from_LEAF_cat_in_last_30_days',
 'f_num_purchases_from_LEAF_cat_in_last_60_days',
 'f_num_purchases_from_LEAF_cat_in_last_90_days',
 'f_num_purchases_from_LEAF_cat_in_last_180_days',
 'f_num_purchases_from_LEAF_cat_in_last_360_days',
 'f_num_purchases_from_LVL2_cat_in_last_1_days',
 'f_num_purchases_

## Sample data for tests

In [6]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np


specific_buyer_ids = [11245860, 1498530061, 1324204268, 1132069035] # 5 rows total with leaf

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))


In [66]:
import feature_helpers as fe

In [25]:
sum ([0.35, 0.25, 0.2, 0.1, 0.06, 0.04])

1.0

In [24]:
sum ([0.35, 0.3, 0.25, 0.2, 0.05, 0.05])

1.2

## Calc weighted actions of user:

In [69]:
from pyspark.sql import functions as F

def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 
    
def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME

def f_weighted_num_actions_from_cat_in_last_x_days(df_base, df_prch, cat_type="LEAF", action_type="purchase",
                                                   time_windows=[2, 7, 30, 90, 180, 360], weights=[0.35, 0.25, 0.2, 0.1, 0.06, 0.04]): 
    
    # Define the final feature name for weighted actions
    feature_name = f"f_weighted_num_{action_type}s_from_{cat_type}_cat"
    
    # Initialize a column to hold the cumulative weighted sum
    df_base = df_base.withColumn(feature_name, F.lit(0))
    
    # Calculate each time-window feature and apply the corresponding weight
    for i, days in enumerate(time_windows):
        # Call `f_num_actions_from_cat_in_last_x_days` for each time window
        df_window = f_num_actions_from_cat_in_last_x_days(df_base, df_prch, cat_type=cat_type, days=days, action_type=action_type)
       
        
        # Generate the specific feature name from the base function for this time window
        window_feature_name = f"f_num_{action_type}s_from_{cat_type}_cat_in_last_{days}_days"
        
        # Rename BUYER_ID and EVENT_TIMESTAMP to avoid ambiguity
        df_window = (df_window
                     .withColumnRenamed("BUYER_ID", "window_BUYER_ID")
                     .withColumnRenamed("EVENT_TIMESTAMP", "window_EVENT_TIMESTAMP"))
        
        # Join the result to the base DataFrame and apply the weight
        df_base = (df_base
                   .join(
                       df_window.select("window_BUYER_ID", _leaf_id_col(df_window, cat_type), "window_EVENT_TIMESTAMP", window_feature_name),
                       on=[
                           df_base["BUYER_ID"] == df_window["window_BUYER_ID"],
                           _leaf_condition(df_base, df_window, cat_type),
                           df_base["EVENT_TIMESTAMP"] == df_window["window_EVENT_TIMESTAMP"]
                       ], 
                       how="left"
                   )
                   .select(
                       df_base["BUYER_ID"],
                       _leaf_id_col(df_base, cat_type),
                       df_base["EVENT_TIMESTAMP"],
                       df_base[feature_name],
                       F.coalesce(F.col(window_feature_name), F.lit(0)).alias(f"weighted_{window_feature_name}")
                   ))
        
        
        
        # If column doesn't exist, use 0 as default and then multiply by the weight
        weighted_column = F.coalesce(F.col(f"weighted_{window_feature_name}"), F.lit(0)) * weights[i]
        
        # Accumulate the weighted sum into the main feature column
        df_base = df_base.withColumn(feature_name, F.col(feature_name) + weighted_column)
      #  df_base.show()
    # Return only necessary columns: identifier and the final weighted feature
    df_res = df_base.select("BUYER_ID", _leaf_id_col(df_base, cat_type), "EVENT_TIMESTAMP", feature_name)
    
    return df_res


In [70]:
df_tsf= f_weighted_num_actions_from_cat_in_last_x_days(df_smp_sample, df_vi_sample, cat_type="LEAF", action_type="purchase",
                                                       time_windows=[2, 7, 30, 90, 180, 360], weights=[0.35, 0.25, 0.2, 0.1, 0.06, 0.04])
    # Define the final feature name for weighted actions
df_tsf.show()

+----------+-------------+---------------+--------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_weighted_num_purchases_from_LEAF_cat|
+----------+-------------+---------------+--------------------------------------+
|1132069035|        24410|     2024-05-01|                                  0.28|
|1498530061|       168135|     2024-05-01|                                   8.0|
|  11245860|       127413|     2024-05-01|                                   0.0|
|  11245860|        47173|     2024-05-01|                                   0.0|
|1324204268|         2885|     2024-05-01|                                   0.1|
+----------+-------------+---------------+--------------------------------------+



In [7]:
import features as F
df_ft = F.f_weighted_num_actions_from_cat_in_last_x_days(df_smp_sample, df_vi_sample, cat_type="LEAF", action_type="purchase", \
                                                  time_windows=[2, 7, 30, 90, 180, 360], weights=[0.35, 0.25, 0.2, 0.1, 0.06, 0.04])
df_ft.show()

+----------+-------------+---------------+--------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_weighted_num_purchases_from_LEAF_cat|
+----------+-------------+---------------+--------------------------------------+
|1132069035|        24410|     2024-05-01|                                  0.28|
|1498530061|       168135|     2024-05-01|                                   8.0|
|  11245860|       127413|     2024-05-01|                                   0.0|
|  11245860|        47173|     2024-05-01|                                   0.0|
|1324204268|         2885|     2024-05-01|                                   0.1|
+----------+-------------+---------------+--------------------------------------+



In [7]:
import features as F
df_ft = F.f_weighted_num_actions_from_cat_in_last_x_days(df_smp_sample, df_vi_sample, cat_type="LVL2", action_type="purchase", \
                                                  time_windows=[2, 7, 30, 90, 180, 360], weights=[0.35, 0.25, 0.2, 0.1, 0.06, 0.04])
df_ft.show()

+----------+-------------+---------------+--------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|f_weighted_num_purchases_from_LVL2_cat|
+----------+-------------+---------------+--------------------------------------+
|1324204268|        24409|     2024-05-01|                                   0.3|
|1132069035|        24409|     2024-05-01|                                  0.48|
|  11245860|       181417|     2024-05-01|                                   0.0|
|1498530061|          631|     2024-05-01|                                131.75|
|  11245860|       181423|     2024-05-01|                                   0.0|
+----------+-------------+---------------+--------------------------------------+



In [ ]:
df_ft = F.f_weighted_num_actions_from_cat_in_last_x_days(df_smp_sample, df_vi_sample, cat_type="META", action_type="purchase", \
                                                  time_windows=[2, 7, 30, 90, 180, 360], weights=[0.35, 0.25, 0.2, 0.1, 0.06, 0.04])
df_ft.show()

# Calculating ratio of actions: views/ purchases

In [71]:
def f_actions_ratio_from_cat_in_last_x_days(df_base, df_vi, df_prch, cat_type="LEAF", days=30, action_types=["view", "purchase"]):
    # Generate dynamic feature names based on action types
    action_1_feature = f"f_num_{action_types[0]}s_from_{cat_type}_cat_in_last_{days}_days"
    action_2_feature = f"f_num_{action_types[1]}s_from_{cat_type}_cat_in_last_{days}_days"
    ratio_feature = f"f_{action_types[0]}_to_{action_types[1]}_ratio_from_{cat_type}_cat_in_last_{days}_days"
    
    # Calculate counts for the first action type 
    df_vi_counts = f_num_actions_from_cat_in_last_x_days(df_base, df_vi, cat_type=cat_type, days=days, action_type=action_types[0])
    
    # Calculate counts for the second action type 
    df_prch_counts = f_num_actions_from_cat_in_last_x_days(df_base, df_prch, cat_type=cat_type, days=days, action_type=action_types[1])
    
    # Join action counts on buyer ID, category ID, and event timestamp
    df_ratio = (df_vi_counts
                .join(df_prch_counts, 
                      on=[df_vi_counts.BUYER_ID == df_prch_counts.BUYER_ID,
                          _leaf_condition(df_vi_counts, df_prch_counts, cat_type),
                          df_vi_counts.EVENT_TIMESTAMP == df_prch_counts.EVENT_TIMESTAMP],
                      how="left")
                .select(df_vi_counts.BUYER_ID, 
                        _leaf_id_col(df_vi_counts, cat_type),
                        df_vi_counts.EVENT_TIMESTAMP,
                        F.coalesce(df_vi_counts[action_1_feature], F.lit(0)).alias(action_1_feature),
                        F.coalesce(df_prch_counts[action_2_feature], F.lit(0)).alias(action_2_feature))
               )
    
    # Calculate the ratio
    df_ratio = df_ratio.withColumn(ratio_feature,
                                   F.when(F.col(action_2_feature) == 0, F.lit(-1))  # Avoid division by zero
                                   .otherwise(F.col(action_1_feature) / F.col(action_2_feature)))
    # Dropping the specified columns
    df_ratio = df_ratio.drop(action_1_feature , action_2_feature)

    
    return df_ratio

In [72]:
df_tsf= f_actions_ratio_from_cat_in_last_x_days(df_smp_sample, df_vi_sample, df_prch_sample, cat_type="LEAF", 
                                                days=360, action_types=["view", "purchase"])
df_tsf.show()

+----------+-------------+---------------+-------------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_view_to_purchase_ratio_from_LEAF_cat_in_last_360_days|
+----------+-------------+---------------+-------------------------------------------------------+
|1132069035|        24410|     2024-05-01|                                                    7.0|
|1498530061|       168135|     2024-05-01|                                                   7.25|
|  11245860|       127413|     2024-05-01|                                                    0.0|
|  11245860|        47173|     2024-05-01|                                                    0.0|
|1324204268|         2885|     2024-05-01|                                                    1.0|
+----------+-------------+---------------+-------------------------------------------------------+



# Normalizing views and purchases:

In [11]:
def f_num_actions_from_cat_in_last_x_days(df_base, df_action, cat_type="LEAF", days=1, action_type="purchase"):
    feature_name = f"f_num_{action_type}s_from_{cat_type}_cat_in_last_{days}_days"
    df_res = (df_base.join(df_action, (df_base.BUYER_ID == df_action.BUYER_ID) & _leaf_condition(df_base, df_action, cat_type) &
                       (F.datediff(df_base.EVENT_TIMESTAMP,df_action.EVENT_TIMESTAMP) > 0) &
                       (F.datediff(df_base.EVENT_TIMESTAMP,df_action.EVENT_TIMESTAMP) <= days), how="left")
                  .groupBy(df_base.BUYER_ID, _leaf_id_col(df_base, cat_type), df_base.EVENT_TIMESTAMP)
                  .agg(F.count(df_action.EVENT_TIMESTAMP).alias(feature_name))
                  .fillna(0, subset=[feature_name])
     )
    return df_res

In [22]:
df_tsf= f_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=360 ,action_type="view")


In [24]:
df_tsf.show()

24/11/03 12:19:16 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_num_views_from_LEAF_cat_in_last_360_days|
+----------+-------------+---------------+------------------------------------------+
|  11245860|        47173|     2024-05-01|                                         3|
|1132069035|        24410|     2024-05-01|                                         1|
|  11245860|       127413|     2024-05-01|                                         1|
|1498530061|       168135|     2024-05-01|                                         4|
|1324204268|         2885|     2024-05-01|                                         1|
+----------+-------------+---------------+------------------------------------------+



In [36]:


def f_norm_num_actions_from_cat_in_last_x_days(df_base, df_action, cat_type="LEAF", days=1, action_type="purchase"):
    # Step 1: Calculate the number of actions from the specific category type
    cat_actions_df = f_num_actions_from_cat_in_last_x_days(df_base, df_action, cat_type=cat_type, days=days, action_type=action_type)
    
    # Update the feature name for category-specific actions
    feature_name = f"f_norm_num_{action_type}s_from_{cat_type}_cat_in_last_{days}_days"
    
    # Step 2: Calculate total actions across all categories of the same type (not limited to a specific leaf)
    total_actions = f"f_total_{action_type}s_in_last_{days}_days" 
    df_total_actions = (df_base.join(df_action, 
                                      (df_base.BUYER_ID == df_action.BUYER_ID) & 
                                      (F.datediff(df_base.EVENT_TIMESTAMP, df_action.EVENT_TIMESTAMP) > 0) &
                                      (F.datediff(df_base.EVENT_TIMESTAMP, df_action.EVENT_TIMESTAMP) <= days), 
                                      how="left")
                        .groupBy(df_base.BUYER_ID, df_base.EVENT_TIMESTAMP)
                        .agg(F.count(df_action.EVENT_TIMESTAMP).alias(total_actions))
                        .fillna(0, subset=[total_actions])
    )

    # Step 3: Join the category-specific actions with the total actions
    df_result = cat_actions_df.join(df_total_actions, 
                                     ["BUYER_ID", "EVENT_TIMESTAMP"], 
                                     "left").fillna(0)

    # Step 4: Normalize the category-specific actions by the total actions
    df_result = df_result.withColumn(feature_name,
                                      F.when(df_result[total_actions] > 0,  
                                              F.round(
                                                  df_result[f"f_num_{action_type}s_from_{cat_type}_cat_in_last_{days}_days"] / df_result[total_actions], 
                                                  5)  
                                              ).otherwise(-1))

    # Step 5: Drop intermediate columns
    df_result = df_result.drop(f"f_num_{action_type}s_from_{cat_type}_cat_in_last_{days}_days", total_actions)

    return df_result


In [37]:
df_tsf= f_norm_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", 
                                                   days=360 ,action_type="view")
df_tsf.show()

+----------+---------------+-------------+-----------------------------------------------+
|  BUYER_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|f_norm_num_views_from_LEAF_cat_in_last_360_days|
+----------+---------------+-------------+-----------------------------------------------+
|  11245860|     2024-05-01|       127413|                                         6.1E-4|
|  11245860|     2024-05-01|        47173|                                        0.00182|
|1324204268|     2024-05-01|         2885|                                        0.04167|
|1498530061|     2024-05-01|       168135|                                        0.03053|
|1132069035|     2024-05-01|        24410|                                        0.00885|
+----------+---------------+-------------+-----------------------------------------------+



In [38]:
df_tsf.select("f_norm_num_views_from_LEAF_cat_in_last_360_days").show()

+-----------------------------------------------+
|f_norm_num_views_from_LEAF_cat_in_last_360_days|
+-----------------------------------------------+
|                                        0.00182|
|                                         6.1E-4|
|                                        0.04167|
|                                        0.03053|
|                                        0.00885|
+-----------------------------------------------+



In [39]:
df_ft = f_norm_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=360 ,action_type="view")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.count())

df_ft = f_norm_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LVL2", days=360 ,action_type="view") 
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.count()) 

df_ft = f_norm_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="META", days=360 ,action_type="view") 
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

5


5


24/11/03 13:03:31 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_norm_num_views_from_LEAF_cat_in_last_360_days|f_norm_num_views_from_LVL2_cat_in_last_360_days|f_norm_num_views_from_META_cat_in_last_360_days|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|  11245860|          260|     2024-05-01|       181423|       127413|              0|    0|Stamps:A

5


In [40]:
df_res.columns

['BUYER_ID',
 'META_CATEG_ID',
 'EVENT_TIMESTAMP',
 'CATEG_LVL2_ID',
 'LEAF_CATEG_ID',
 'num_repurchases',
 'label',
 'LEAF_CATEG_NAME',
 'CATEG_LVL2_NAME',
 'META_CATEG_NAME',
 'f_norm_num_views_from_LEAF_cat_in_last_360_days',
 'f_norm_num_views_from_LVL2_cat_in_last_360_days',
 'f_norm_num_views_from_META_cat_in_last_360_days']

In [42]:
df_res.select(['BUYER_ID','LEAF_CATEG_ID', 'label' ,
               'f_norm_num_views_from_LEAF_cat_in_last_360_days',
               'f_norm_num_views_from_LVL2_cat_in_last_360_days']).show()

+----------+-------------+-----+-----------------------------------------------+-----------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|label|f_norm_num_views_from_LEAF_cat_in_last_360_days|f_norm_num_views_from_LVL2_cat_in_last_360_days|
+----------+-------------+-----+-----------------------------------------------+-----------------------------------------------+
|  11245860|       127413|    0|                                         6.1E-4|                                        0.02005|
|  11245860|        47173|    0|                                        0.00182|                                        0.00547|
|1498530061|       168135|    1|                                        0.03053|                                         0.8626|
|1324204268|         2885|    0|                                        0.04167|                                        0.04167|
|1132069035|        24410|    0|                                        0.00885|                 

In [ ]:
df_ft = F.f_norm_num_actions_from_cat_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=1, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

# User behaviour features:

## time_since_first_view_after_purchase_in_cat 

In [7]:
def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 
    
def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME

#----------------------------------------------------------------------
from pyspark.sql import functions as F
from pyspark.sql import Window  
import feature_helpers as fe

def _first_view_after_purchase(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Combined Step 1 and Step 2: Join purchases and views, filter, group by, and aggregate
    df_res = (df_prch.join(df_vi, 
                           (df_prch.BUYER_ID == df_vi.BUYER_ID) & 
                           _leaf_condition(df_prch, df_vi, cat_type), 
                           how="left")
                      .withColumn("time_diff", F.datediff(df_vi.EVENT_TIMESTAMP, df_prch.EVENT_TIMESTAMP))  # Calculate time diff
                      .filter(F.col("time_diff") > 0)  # Keep only future views
                      .groupBy(
                          df_prch.BUYER_ID,  
                          _leaf_id_col(df_prch, cat_type), 
                          df_prch.EVENT_TIMESTAMP.alias("purchase_timestamp")
                      )
                      .agg(
                          F.min(df_vi.EVENT_TIMESTAMP).alias("first_view_timestamp"),  # Get the minimum view timestamp
                          F.min("time_diff").alias("days_diff")
                      )
                      .filter(F.col("first_view_timestamp").isNotNull())
              )

    # Step 3: Use window function to ensure only the closest view is kept
    window_spec = Window.partitionBy(
                          df_res.BUYER_ID, 
                          _leaf_id_col(df_res, cat_type),
                          df_res.first_view_timestamp  
                      ).orderBy(
                          F.col("days_diff").asc()  
                      )  

    # Rank based on the closest purchase to each view timestamp
    df_res = (df_res.withColumn("rank", F.row_number().over(window_spec))  # Rank rows based on days_diff
              .filter(F.col("rank") == 1)  # Keep only the closest view (rank == 1)
              .drop("rank")  
              )

    # Step 4: Join with df_base and ensure base.EVENT_TIMESTAMP > first_view_timestamp
    df_res = (df_res.join(df_base, 
                      (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  
                      _leaf_condition(df_res, df_base, cat_type), 
                      how="inner")  # Use inner join to filter based on conditions
          .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_res.first_view_timestamp) > 0)  # Filter rows based on date difference
          .select(
              df_res.BUYER_ID,
              _leaf_id_col(df_res, cat_type),
              df_res.purchase_timestamp,
              df_res.first_view_timestamp,
              df_res.days_diff
                 )
              )

    # Step 5: Collect all days_diff values into an array for aggregation
    df_res = (df_res.groupBy(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type))
                     .agg(F.collect_list("days_diff").alias("days_diff_array"))  # Collect all days_diff into an array
             )
    return df_res 
def time_since_first_view_after_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the first view after each purchase
    df_first_view = _first_view_after_purchase(df_prch, df_vi,df_base, cat_type)
   
    # Step 2: Join df_base with df_first_view to get days_diff_array
    df_days_diff = (
        df_base
        .join(
            df_first_view,
            (df_base.BUYER_ID == df_first_view.BUYER_ID) & _leaf_condition(df_base, df_first_view, cat_type),
            how="left"
        )
        .select(
            df_base.BUYER_ID,
            _leaf_id_col(df_base, cat_type),
            df_base.EVENT_TIMESTAMP,
            F.col("days_diff_array")
        )
    )
    df_days_diff.printSchema()  
    df_days_diff.show(5) 
        
    df_days_diff_agg = (
        df_days_diff
        .groupBy(
            df_days_diff.BUYER_ID,
            _leaf_id_col(df_days_diff, cat_type),
            df_days_diff.EVENT_TIMESTAMP
        )
        .agg(
            F.min(F.when(F.col("days_diff_array").isNull(), F.array(F.lit(-1))).otherwise(F.col("days_diff_array"))).alias(f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat"),
            F.max(F.when(F.col("days_diff_array").isNull(), F.array(F.lit(-1))).otherwise(F.col("days_diff_array"))).alias(f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat"),
            fe.array_mean(F.when(F.col("days_diff_array").isNull(),F.array(F.lit(-1))).otherwise(F.col("days_diff_array"))).alias(f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat")
        )
        .fillna(-1, subset=[f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                        f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                        f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat"
                           ])
                .drop("days_diff_array") )

    return df_res 


def add_feature(df_base, df_feature, feature_name=None, join_key="ITEM_ID", na_value=None, how="inner"):
    df_res = df_base.join(df_feature, on=["BUYER_ID", join_key, "EVENT_TIMESTAMP"], how=how)
    if na_value:
        df_res = df_res.fillna(na_value, subset=[feature_name])
    return df_res

In [ ]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np


specific_buyer_ids = [11245860, 1498530061, 1324204268, 1132069035]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))


In [51]:
#df_smp_sample.show()

+-------------+----------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+
|LEAF_CATEG_ID|  BUYER_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|
+-------------+----------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+
|       168135|1498530061|     2024-05-01|              6|    1|Home & Garden:Too...|          631|Tools & Workshop ...|        11700|       Home & Garden|
|        47173|  11245860|     2024-05-01|              0|    0|Stamps:Latin Amer...|       181417|       Latin America|          260|              Stamps|
|        24410|1132069035|     2024-05-01|              0|    0|Sports Mem, Cards...|        24409|Fan Apparel & Sou...|        64482|Sports Mem, Cards...|
|         2885|1324204268|     2024-05-01|              0|    0|

In [12]:
#df_smp_sample.count()

5

In [47]:
def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 

def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME  
    
def _first_view_after_purchase(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Combined Step 1 and Step 2: Join purchases and views, filter, group by, and aggregate
    df_res = (df_prch.join(df_vi, 
                           (df_prch.BUYER_ID == df_vi.BUYER_ID) & _leaf_condition(df_prch, df_vi, cat_type), how="left")
                      .withColumn("time_diff", F.datediff(df_vi.EVENT_TIMESTAMP, df_prch.EVENT_TIMESTAMP)).filter(F.col("time_diff") > 0)
                      .groupBy(df_prch.BUYER_ID, _leaf_id_col(df_prch, cat_type), df_prch.EVENT_TIMESTAMP.alias("purchase_timestamp"))
                      .agg(
                          F.min(df_vi.EVENT_TIMESTAMP).alias("first_view_timestamp"),
                          F.min("time_diff").alias("days_diff")
                      )
                      .filter(F.col("first_view_timestamp").isNotNull()))

    # Step 3: Use window function to ensure only the closest view is kept - same view can be assigned to multiple purchases so we take the closest purchase by days diff 
    window_spec = Window.partitionBy( df_res.BUYER_ID, _leaf_id_col(df_res, cat_type), df_res.first_view_timestamp).orderBy(F.col("days_diff").asc())  
    # Rank based on the closest purchase to each view timestamp
    df_res = (df_res.withColumn("rank", F.row_number().over(window_spec)).filter(F.col("rank") == 1)  # Keep only the closest view (rank == 1)
              .drop("rank"))

    # Step 4: Join with df_base and ensure base.EVENT_TIMESTAMP > first_view_timestamp
    df_res = (df_res.join(df_base, (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  _leaf_condition(df_res, df_base, cat_type), how="inner")
          .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_res.first_view_timestamp) > 0)
          .select(df_res.BUYER_ID,_leaf_id_col(df_res, cat_type), df_res.purchase_timestamp, df_res.first_view_timestamp,df_res.days_diff))

    # Step 5: Collect all days_diff values into an array for aggregation
    df_res = (df_res.groupBy(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type))
                     .agg(F.collect_list("days_diff").alias("days_diff_array")) )

    #df_res.show()
    return df_res  


def time_since_first_view_after_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the first view after each purchase
    df_first_view = _first_view_after_purchase(df_prch, df_vi, df_base, cat_type)
    
    # Step 2: Calculate the days_diff_array in a separate DataFrame
    df_days_diff = (df_base.join(df_first_view, 
                                  (df_base.BUYER_ID == df_first_view.BUYER_ID) & 
                                  _leaf_condition(df_base, df_first_view, cat_type), how="left")
                     .select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type), 
                             df_base.EVENT_TIMESTAMP,
                             F.col("days_diff_array").alias("days_diff_view"))
                    )

    df_days_diff.printSchema() 
    df_res = (df_days_diff.groupBy(df_days_diff.BUYER_ID, 
                                _leaf_id_col(df_days_diff, cat_type), 
                                df_days_diff.EVENT_TIMESTAMP)
                 .agg(
                     F.array_min(F.col("days_diff_view")).alias(f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat"),
                     F.array_max(F.col("days_diff_view")).alias(f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat"),
                     fe.array_mean(F.col("days_diff_view")).alias(f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat")
                 )
                 .fillna(-1, subset=[
                     f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                     f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                     f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat"
                 ])
          )

    return df_res 

In [90]:
def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 

def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME 
    
def _first_view_after_purchase(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Combined Step 1 and Step 2: Join purchases and views, filter, group by, and aggregate
    df_res = (df_prch.join(df_vi, 
                           (df_prch.BUYER_ID == df_vi.BUYER_ID) & _leaf_condition(df_prch, df_vi, cat_type), how="left")
                      .withColumn("time_diff", F.datediff(df_vi.EVENT_TIMESTAMP, df_prch.EVENT_TIMESTAMP)).filter(F.col("time_diff") > 0)
                      .groupBy(df_prch.BUYER_ID, _leaf_id_col(df_prch, cat_type), df_prch.EVENT_TIMESTAMP.alias("purchase_timestamp"))
                      .agg(
                          F.min(df_vi.EVENT_TIMESTAMP).alias("first_view_timestamp"),
                          F.min("time_diff").alias("days_diff")
                      )
                      .filter(F.col("first_view_timestamp").isNotNull()))

    # Step 3: Use window function to ensure only the closest view is kept - same view can be assigned to multiple purchases so we take the closest purchase by days diff 
    window_spec = Window.partitionBy( df_res.BUYER_ID, _leaf_id_col(df_res, cat_type), df_res.first_view_timestamp).orderBy(F.col("days_diff").asc())  
    # Rank based on the closest purchase to each view timestamp
    df_res = (df_res.withColumn("rank", F.row_number().over(window_spec)).filter(F.col("rank") == 1)  # Keep only the closest view (rank == 1)
              .drop("rank"))

    # Step 4: Join with df_base and ensure base.EVENT_TIMESTAMP > first_view_timestamp
    df_res = (df_res.join(df_base, (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  _leaf_condition(df_res, df_base, cat_type), how="inner")
          .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_res.first_view_timestamp) > 0)
          .select(df_res.BUYER_ID,_leaf_id_col(df_res, cat_type), df_res.purchase_timestamp, df_res.first_view_timestamp,df_res.days_diff))

    # Step 5: Collect all days_diff values into an array for aggregation
    df_res = (df_res.groupBy(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type))
                     .agg(F.collect_list("days_diff").alias("days_diff_array")) )

   # df_res.show()
    return df_res 

def time_since_first_view_after_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the first view after each purchase
    df_first_view = _first_view_after_purchase(df_prch, df_vi,df_base, cat_type)
    

    # Step 2: Join df_base with df_first_view to get days_diff_array
    df_days_diff = (df_base.join(df_first_view, 
                                  (df_base.BUYER_ID == df_first_view.BUYER_ID) & 
                                  _leaf_condition(df_base, df_first_view, cat_type) , how="left")
                     .select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type), 
                             df_base.EVENT_TIMESTAMP,
                             F.col("days_diff_array"))  
                    )
    df_days_diff.show() 
    
    cat_col= _leaf_id_col(df_days_diff, cat_type)
    
    df_days_diff = df_days_diff.dropDuplicates(
        subset=[df_days_diff.BUYER_ID, cat_col, df_days_diff.EVENT_TIMESTAMP]
    ) 
    
    # Step 3: Calculate features based on days_diff_array (min, max, mean)
    df_res = (df_days_diff.withColumn(f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat", F.array_min(F.col("days_diff_array")))  
                         .withColumn(f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat", F.array_max(F.col("days_diff_array")))  
                         .withColumn(f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat", fe.array_mean(F.col("days_diff_array")))   
                         .fillna(-1, subset=[f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                                             f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                                             f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat"])
                         .drop("days_diff_array") 
            )

    return df_res


#### time since first view grouped

In [8]:
def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID) 

def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME 

In [14]:
from pyspark.sql import functions as F
from pyspark.sql import Window

def _first_view_after_purchase(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Combined Step 1 and Step 2: Join purchases and views, filter, group by, and aggregate
    df_res = (df_prch.join(df_vi, 
                           (df_prch.BUYER_ID == df_vi.BUYER_ID) & _leaf_condition(df_prch, df_vi, cat_type), how="left")
                      .withColumn("time_diff", F.datediff(df_vi.EVENT_TIMESTAMP, df_prch.EVENT_TIMESTAMP)).filter(F.col("time_diff") > 0)
                      .groupBy(df_prch.BUYER_ID, _leaf_id_col(df_prch, cat_type), df_prch.EVENT_TIMESTAMP.alias("purchase_timestamp"))
                      .agg(
                          F.min(df_vi.EVENT_TIMESTAMP).alias("first_view_timestamp"),
                          F.min("time_diff").alias("days_diff")
                      )
                      .filter(F.col("first_view_timestamp").isNotNull()))

    # Step 3: Use window function to ensure only the closest view is kept - same view can be assigned to multiple purchases so we take the closest purchase by days diff 
    window_spec = Window.partitionBy(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type), df_res.first_view_timestamp).orderBy(F.col("days_diff").asc())  
    # Rank based on the closest purchase to each view timestamp
    df_res = (df_res.withColumn("rank", F.row_number().over(window_spec)).filter(F.col("rank") == 1)  # Keep only the closest view (rank == 1)
              .drop("rank"))

    # Step 4: Join with df_base and ensure base.EVENT_TIMESTAMP > first_view_timestamp
    df_res = (df_res.join(df_base, (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  _leaf_condition(df_res, df_base, cat_type), how="inner")
              .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_res.first_view_timestamp) > 0)
              .select(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type), df_res.purchase_timestamp, df_res.first_view_timestamp, df_res.days_diff))

    return df_res  

def time_since_first_view_after_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the first view after each purchase
    df_first_view = _first_view_after_purchase(df_prch, df_vi, df_base, cat_type)
    
    # Step 2: Join with df_base and perform aggregation directly, calculating min, max, and mean days_diff in a single step
    df_res = df_base.join(df_first_view, 
                           (df_base.BUYER_ID == df_first_view.BUYER_ID) & 
                           _leaf_condition(df_base, df_first_view, cat_type), how="left").select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type), 
                             df_base.EVENT_TIMESTAMP,
                             F.col("days_diff"))  
  
    
    df_res_agg= df_res.groupBy(df_res.BUYER_ID, 
                       _leaf_id_col(df_res, cat_type), 
                       df_base.EVENT_TIMESTAMP).agg(
                  F.array_min(F.collect_list("days_diff")).alias(f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat"),
                  F.array_max(F.collect_list("days_diff")).alias(f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat"),
                  fe.array_mean(F.collect_list("days_diff")).alias(f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat")
              ).fillna(-1, subset=[
                  f"f_min_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                  f"f_max_time_since_first_view_after_purchase_in_{cat_type}_cat", 
                  f"f_mean_time_since_first_view_after_purchase_in_{cat_type}_cat"
              ])
          
    
    
    return df_res_agg


In [10]:
df_tsf= time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

In [11]:
df_tsf.show()

+----------+-------------+---------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|
+----------+-------------+---------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  11245860|        47173|     2024-05-01|                                                    -1|                                                    -1|                                                   -1.0|
|1132069035|        24410|     2024-05-01|                                                    -1|                                                    -1|            

In [77]:
df_tsf= time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

7


5


+----------+-------------+---------------+---------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|days_diff|
+----------+-------------+---------------+---------+
|  11245860|        47173|     2024-05-01|     null|
|1132069035|        24410|     2024-05-01|     null|
|  11245860|       127413|     2024-05-01|     null|
|1498530061|       168135|     2024-05-01|       76|
|1498530061|       168135|     2024-05-01|       21|
|1498530061|       168135|     2024-05-01|        7|
|1324204268|         2885|     2024-05-01|     null|
+----------+-------------+---------------+---------+

None


5


In [78]:
df_tsf.show()

+----------+-------------+---------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|
+----------+-------------+---------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  11245860|        47173|     2024-05-01|                                                    -1|                                                    -1|                                                   -1.0|
|1132069035|        24410|     2024-05-01|                                                    -1|                                                    -1|            

In [17]:
def add_feature(df_base, df_feature, feature_name=None, join_key="ITEM_ID", na_value=None, how="inner"):
    df_res = df_base.join(df_feature, on=["BUYER_ID", join_key, "EVENT_TIMESTAMP"], how=how)
   # if na_value:
    if na_value is not None and feature_name:
        df_res = df_res.fillna(na_value, subset=[feature_name])
    return df_res

##### time since first view - adding by cat

In [18]:

df_ft = time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

df_ft = time_since_first_view_after_purchase_in_cat(df_smp, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = time_since_first_view_after_purchase_in_cat(df_smp, df_prch_sample, df_vi_sample, cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|1132069035|        24410|     202

24/10/28 17:50:31 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

5


+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|f_min_time_since_first_view_after_purchase_in_LVL2_cat|f_max_time_since_first_view_after_purchase_in_LVL2_cat|f_mean_time_since_first_view_after_purchase_in_LVL2_cat|
+----------+------------

5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|f_min_time_since_first_vi

5


In [19]:
df_res.show()

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|f_min_time_since_first_vi

In [ ]:
df_tsf.select(["f_max_time_since_first_view_after_purchase_in_LEAF_cat",
               "f_mean_time_since_first_view_after_purchase_in_LEAF_cat",
               "f_min_time_since_first_view_after_purchase_in_LEAF_cat"]).show()

In [21]:
df_tsf.count()

5

##### time since first view - sample features adding F. 

In [4]:
import features as F

# Step 1: Sample 20 buyers from df_smp
sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(1000)).collect()
buyer_ids = [row["BUYER_ID"] for row in sample_buyers]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those 20 buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(buyer_ids))
print(df_smp_sample.count())
# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")
print(df_res_t.count())
# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")
print(df_res_t.count())
# Step 5: Calculate features for META category
df_ft_meta = F.f_time_since_first_view_after_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")
print(df_res_t.count())
# Step 6: Show the results
df_res_t.show()

1006


1006


1006


1006


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_min_time_since_first_view_after_purchase_in_LEAF_cat|f_max_time_since_first_view_after_purchase_in_LEAF_cat|f_mean_time_since_first_view_after_purchase_in_LEAF_cat|f_min_time_since_first_vi

## f_num_views_from_purchase_in_cat 

In [3]:


def _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Step 1: Get unique purchase timestamps per buyer and category
    purchase_df = df_prch.select("BUYER_ID", _leaf_id_col(df_prch, cat_type), "EVENT_TIMESTAMP").distinct()
    
    # Step 2: Create a window to identify the next purchase timestamp for each buyer and category
    window_spec = Window.partitionBy("BUYER_ID", _leaf_id_col(df_prch, cat_type)).orderBy(F.col("EVENT_TIMESTAMP").asc())
    
    # Add the next purchase timestamp using lead
    purchase_df = purchase_df.withColumn("next_purchase_timestamp", F.lead("EVENT_TIMESTAMP", 1).over(window_spec))

    # Step 3: Join views with purchases to find views that fall between purchases
    view_purchase_joined = df_vi.join(
        purchase_df,
        (df_vi.BUYER_ID == purchase_df.BUYER_ID) &
        _leaf_condition(purchase_df, df_vi, cat_type),  # Ensure the condition uses purchase_df
        how="left"
    ).select(
        df_vi.BUYER_ID,
        df_vi.ITEM_ID,
        df_vi.EVENT_TIMESTAMP.alias("view_timestamp"),  # Rename view timestamp for clarity
        purchase_df.EVENT_TIMESTAMP.alias("purchase_timestamp"),  # Rename purchase timestamp for clarity
        _leaf_id_col(purchase_df, cat_type),  # Include leaf category ID from purchase_df
        purchase_df.next_purchase_timestamp
    )
    print("step 1") 
    view_purchase_joined.show()
    
    # Step 4: Filter views to ensure each view is associated with the closest past purchase
    view_purchase_filtered = view_purchase_joined.filter(
        (view_purchase_joined.view_timestamp > view_purchase_joined.purchase_timestamp) & 
        (view_purchase_joined.view_timestamp <= F.coalesce(view_purchase_joined.next_purchase_timestamp, F.lit('9999-12-31')))
    )
    print("step 4 Filter views by dt") 
    view_purchase_filtered.show()
    
    # Step 5: Use a window function to rank views per buyer and category and purchase 
    view_ranked = view_purchase_filtered.withColumn(
        "rank",
        F.row_number().over(Window.partitionBy(view_purchase_filtered.BUYER_ID, _leaf_id_col(view_purchase_filtered, cat_type),
                                               view_purchase_filtered.view_timestamp )
                             .orderBy(F.col("purchase_timestamp").desc()))  # Rank by the closest purchase
    )
    print("step 5 rank")
    view_ranked.show()
    
    # Step 6: Keep only the highest-ranked view (the closest to the purchase)
    view_single_assigned = view_ranked.filter(F.col("rank") == 1)
    print("step 6 filter rank")
    view_single_assigned.show()
    
    # Step 7: Select the desired columns and add time_diff
    df_res = view_single_assigned.select(
        "BUYER_ID",
        "ITEM_ID",
        _leaf_id_col(view_single_assigned, cat_type),
        F.col("purchase_timestamp"),
        F.col("view_timestamp"),
        F.datediff(F.col("view_timestamp"), F.col("purchase_timestamp")).alias("time_diff")
    )
    print("step 7 after filter rank")
    df_res.show()

    # Step 8: Join with df_base and select the relevant columns
    df_res = (df_res.join(df_base, 
                          (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  
                          _leaf_condition(df_res, df_base, cat_type), 
                          how="inner")  # Use inner join to filter based on conditions
                  .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_res.view_timestamp) > 0)  # Filter rows based on date difference
                  .select(
                      df_res.BUYER_ID,
                      _leaf_id_col(df_res, cat_type),
                      df_res.purchase_timestamp,
                      df_res.view_timestamp,  # Select view timestamp without alias
                      df_res.time_diff.alias("days_diff")  # Alias for time difference
                  )
              )
    print("step 7 bs filter dt")
    df_res.show()

    # Step 9: Count the views between purchases
    df_view_count = df_res.groupBy("BUYER_ID", _leaf_id_col(df_res, cat_type), "purchase_timestamp") \
                          .agg(F.count(F.col("view_timestamp")).alias("view_count"),
                               F.max(F.col("days_diff")).alias("max_days_diff"))
    df_view_count.show()
    
    # Step 10: Collect all view counts into an array per buyer and category
    df_view_count_array = df_view_count.groupBy("BUYER_ID", _leaf_id_col(df_view_count, cat_type)) \
                                       .agg(F.collect_list(F.col("view_count")).alias("view_count_array"))
    df_view_count_array.show()
    return df_view_count_array


# Function to add the features to the base dataframe
def f_num_views_from_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the number of views between purchases for the given category type
    df_view_counts = _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type)

    # Step 2: Join df_base with df_view_counts to add the new features
    df_res = (df_base.join(df_view_counts, 
                           (df_base.BUYER_ID == df_view_counts.BUYER_ID) & 
                           _leaf_condition(df_base, df_view_counts, cat_type), 
                           how="left")
                     .select(df_base.BUYER_ID, 
                             df_base.LEAF_CATEG_ID,
                             df_base.EVENT_TIMESTAMP,
                             F.col("view_count_array"))  # Include the collected view counts
             )
    print("# Step 2: Join df_base with df_view_counts to add the new features")
    df_res.show()
    
    # Step 3: Calculate features based on view counts (min, max, mean)
    df_res = (df_res.withColumn(f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_array")))  # Min value of the array
                     .withColumn(f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_array")))  # Max value of the array
                     .withColumn(f"f_mean_num_views_between_purchases_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_array")))  # Mean value of the array
                     .fillna(-1, subset=[f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_mean_num_views_between_purchases_in_{cat_type}_cat"])  # Fill nulls with -1
                     .drop("view_count_array")  # Drop the view_count_array if no longer needed
          )

    return df_res


#### prev logic in features with max days: 

In [ ]:
# Function to calculate the number of views between purchases and max days
def _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type="LEAF"):
    
    purchase_df = df_prch.select("BUYER_ID", _leaf_id_col(df_prch, cat_type), "EVENT_TIMESTAMP").distinct()
    window_spec = Window.partitionBy("BUYER_ID", _leaf_id_col(df_prch, cat_type)).orderBy(F.col("EVENT_TIMESTAMP").asc())
    purchase_df = purchase_df.withColumn("next_purchase_timestamp", F.lead("EVENT_TIMESTAMP", 1).over(window_spec))
   
    # Step 3: Join views with purchases to find views that fall between purchases
    view_purchase_joined = df_vi.join(
        purchase_df,
        (df_vi.BUYER_ID == purchase_df.BUYER_ID) &_leaf_condition(purchase_df, df_vi, cat_type), how="left"
        ).select(df_vi.BUYER_ID,df_vi.ITEM_ID,df_vi.EVENT_TIMESTAMP.alias("view_timestamp"),
                 purchase_df.EVENT_TIMESTAMP.alias("purchase_timestamp"), 
        _leaf_id_col(purchase_df, cat_type),  purchase_df.next_purchase_timestamp )
    
    view_purchase_filtered = view_purchase_joined.filter((view_purchase_joined.view_timestamp > view_purchase_joined.purchase_timestamp)& 
        (view_purchase_joined.view_timestamp <= F.coalesce(view_purchase_joined.next_purchase_timestamp, F.lit('9999-12-31'))))
    
    # Step 5: Use a window function to rank views per buyer and category and purchase -- a specific view associated to 1 purchase.
    view_ranked = view_purchase_filtered.withColumn( "rank",F.row_number().over(Window.partitionBy(view_purchase_filtered.BUYER_ID,
                            _leaf_id_col(view_purchase_filtered, cat_type),
                            view_purchase_filtered.view_timestamp ).orderBy(F.col("purchase_timestamp").desc())) )

    # Step 6: Keep only the highest-ranked view (the closest to the purchase)
    view_single_assigned = view_ranked.filter(F.col("rank") == 1)
    
    # Step 7: Select the desired columns and add time_diff
    df_res = view_single_assigned.select("BUYER_ID", "ITEM_ID",_leaf_id_col(view_single_assigned, cat_type), F.col("purchase_timestamp"),
        F.col("view_timestamp"),
        F.datediff(F.col("view_timestamp"), F.col("purchase_timestamp")).alias("time_diff")
    )
    # Step 8: Join with df_base and select the relevant columns
    df_res = (df_res.join(df_base, 
                          (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  _leaf_condition(df_res, df_base, cat_type),  how="inner")  
                  .filter((F.datediff(df_base.EVENT_TIMESTAMP, df_res.view_timestamp) > 0) & 
                    (F.datediff(df_base.EVENT_TIMESTAMP, df_res.purchase_timestamp) > 0)) 
              # Filter rows based on date difference
                  .select(df_res.BUYER_ID, _leaf_id_col(df_res, cat_type),df_res.purchase_timestamp,
                          df_res.view_timestamp, df_res.time_diff.alias("days_diff")  
                  ))    
    # Step 9: Count the views between purchases and calculate max days
    df_view_count = df_res.groupBy("BUYER_ID", _leaf_id_col(df_res, cat_type), "purchase_timestamp") \
                          .agg(F.count(F.col("view_timestamp")).alias("view_count"),F.max(F.col("days_diff")).alias("max_days_diff"))

    # Step 10: Add new feature (views / max_days_diff)
    df_view_count = df_view_count.withColumn( "view_count_per_max_days",
                                         F.when(F.col("max_days_diff") > 0, F.col("view_count") / F.col("max_days_diff")).otherwise(0))
    # Step 11: Collect all view counts into an array per buyer and category
    df_view_count_array = df_view_count.groupBy("BUYER_ID", _leaf_id_col(df_view_count, cat_type)) \
                                       .agg(F.collect_list(F.col("view_count")).alias("view_count_array"),
                                            F.collect_list(F.col("view_count_per_max_days")).alias("view_count_per_max_days_array"))    
    return df_view_count_array


# Function to add the features to the base dataframe
def f_num_views_from_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the number of views between purchases for the given category type
    df_view_counts = _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type)

    # Step 2: Join df_base with df_view_counts to add the new features
    df_res = (df_base.join(df_view_counts, 
                           (df_base.BUYER_ID == df_view_counts.BUYER_ID) & 
                           _leaf_condition(df_base, df_view_counts, cat_type), 
                           how="left")
                     .select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type),
                             df_base.EVENT_TIMESTAMP,
                             F.col("view_count_array"),  # Include the collected view counts
                             F.col("view_count_per_max_days_array"))  # Include the collected views per max days
             )
    
    # Step 3: Calculate features based on view counts (min, max, mean)
    df_res = (df_res.withColumn(f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_array")))  # Min value of the array
                     .withColumn(f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_array")))  # Max value of the array
                     .withColumn(f"f_mean_num_views_between_purchases_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_array")))  # Mean value of the array
                     # Compute min, max, mean for 'view_count_per_max_days_array'
                     .withColumn(f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_per_max_days_array")))  # Min of views/max_days ratio
                     .withColumn(f"f_max_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_per_max_days_array")))  # Max of views/max_days ratio
                     .withColumn(f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_per_max_days_array")))  # Mean of views/max_days ratio
                     # Fill null values with -1
                     .fillna(0, subset=[f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_mean_num_views_between_purchases_in_{cat_type}_cat",
                                         f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                                         f"f_max_views_between_purchases_per_days_in_{cat_type}_cat",
                                         f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat"])  # Fill nulls with -1
                     # Drop the arrays if no longer needed
                     .drop("view_count_array", "view_count_per_max_days_array")
          )

    return df_res


### additional feature of views per days 

In [24]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.sql import Window
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.sql import Window   

def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID)  

def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME

# Function to calculate the number of views between purchases and max days
def _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Step 1: Get unique purchase timestamps per buyer and category
    purchase_df = df_prch.select("BUYER_ID", _leaf_id_col(df_prch, cat_type), "EVENT_TIMESTAMP").distinct()
    
    # Step 2: Create a window to identify the next purchase timestamp for each buyer and category
    window_spec = Window.partitionBy("BUYER_ID", _leaf_id_col(df_prch, cat_type)).orderBy(F.col("EVENT_TIMESTAMP").asc())
    
    # Add the next purchase timestamp using lead
    purchase_df = purchase_df.withColumn("next_purchase_timestamp", F.lead("EVENT_TIMESTAMP", 1).over(window_spec))

    # Step 3: Join views with purchases to find views that fall between purchases
    view_purchase_joined = df_vi.join(
        purchase_df,
        (df_vi.BUYER_ID == purchase_df.BUYER_ID) &
        _leaf_condition(purchase_df, df_vi, cat_type),  # Ensure the condition uses purchase_df
        how="left"
    ).select(
        df_vi.BUYER_ID,
        df_vi.ITEM_ID,
        df_vi.EVENT_TIMESTAMP.alias("view_timestamp"),  # Rename view timestamp for clarity
        purchase_df.EVENT_TIMESTAMP.alias("purchase_timestamp"),  # Rename purchase timestamp for clarity
        _leaf_id_col(purchase_df, cat_type),  # Include leaf category ID from purchase_df
        purchase_df.next_purchase_timestamp
    )
    

    # Step 4: Filter views to ensure each view is associated with the closest past purchase
    view_purchase_filtered = view_purchase_joined.filter(
        (view_purchase_joined.view_timestamp > view_purchase_joined.purchase_timestamp) & 
        (view_purchase_joined.view_timestamp <= F.coalesce(view_purchase_joined.next_purchase_timestamp, F.lit('9999-12-31')))
    )
    
    # Step 5: Use a window function to rank views per buyer and category and purchase -- a specific view associated to 1 purchase.
    view_ranked = view_purchase_filtered.withColumn(
        "rank",
        F.row_number().over(Window.partitionBy(view_purchase_filtered.BUYER_ID, _leaf_id_col(view_purchase_filtered, cat_type),
                                               view_purchase_filtered.view_timestamp )
                             .orderBy(F.col("purchase_timestamp").desc()))  # Rank by the closest purchase
    )

    # Step 6: Keep only the highest-ranked view (the closest to the purchase)
    view_single_assigned = view_ranked.filter(F.col("rank") == 1)
    
    # Step 7: Select the desired columns and add time_diff
    df_res = view_single_assigned.select(
        "BUYER_ID",
        "ITEM_ID",
        _leaf_id_col(view_single_assigned, cat_type),
        F.col("purchase_timestamp"),
        F.col("view_timestamp"),
        F.datediff(F.col("view_timestamp"), F.col("purchase_timestamp")).alias("time_diff")
    )

    # Step 8: Join with df_base and select the relevant columns
    df_res = (df_res.join(df_base, 
                          (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  
                          _leaf_condition(df_res, df_base, cat_type), 
                          how="inner")  # Use inner join to filter based on conditions
                  .filter((F.datediff(df_base.EVENT_TIMESTAMP, df_res.view_timestamp) > 0) & 
                    (F.datediff(df_base.EVENT_TIMESTAMP, df_res.purchase_timestamp) > 0)) 
              # Filter rows based on date difference
                  .select(
                      df_res.BUYER_ID,
                      _leaf_id_col(df_res, cat_type),
                      df_base.EVENT_TIMESTAMP,
                      df_res.purchase_timestamp,
                      df_res.view_timestamp,  # Select view timestamp without alias
                      df_res.time_diff.alias("days_diff")  # Alias for time difference
                  )
              )

    
    # Step 9: Count the views between purchases and calculate max days
    df_view_count = df_res.groupBy("BUYER_ID", _leaf_id_col(df_res, cat_type),"EVENT_TIMESTAMP", "purchase_timestamp") \
                          .agg(F.count(F.col("view_timestamp")).alias("view_count"),
                               F.max(F.col("days_diff")).alias("max_days_diff"))

    # Step 10: Add new feature (views / max_days_diff)

    df_view_count = df_view_count.withColumn(
    "view_count_per_max_days",
    F.when(F.col("max_days_diff") != 0, F.col("view_count") / F.col("max_days_diff")).otherwise(F.lit(0))
)

    #print( df_view_count.show() )
    # Step 11: Collect all view counts into an array per buyer and category
    df_view_count_array = df_view_count.groupBy("BUYER_ID", _leaf_id_col(df_view_count, cat_type),"EVENT_TIMESTAMP") \
                                       .agg(F.collect_list(F.col("view_count")).alias("view_count_array"),
                                            F.collect_list(F.col("view_count_per_max_days")).alias("view_count_per_max_days_array"))
    
    
    print("step of Collect all view counts into an array per buyer and category")
    print( df_view_count_array.count() )
    print( df_view_count_array.show() )
    return df_view_count_array

# Function to add the features to the base dataframe
def f_num_views_from_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the number of views between purchases for the given category type
    df_view_counts = _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type)

    # Step 2: Join df_base with df_view_counts to add the new features
    df_res = (df_base.join(df_view_counts, 
                           (df_base.BUYER_ID == df_view_counts.BUYER_ID) & 
                           (df_base.EVENT_TIMESTAMP == df_view_counts.EVENT_TIMESTAMP)& 
                           _leaf_condition(df_base, df_view_counts, cat_type), 
                           how="left")
                     .select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type),
                             df_base.EVENT_TIMESTAMP,
                             F.col("view_count_array"),  # Include the collected view counts
                             F.col("view_count_per_max_days_array"))  # Include the collected views per max days
             )
    
    print( df_res.count() )
    print( df_res.show() )
    # Step 3: Calculate features based on view counts (min, max, mean)
    df_res_features = (df_res.withColumn(f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_array")))  # Min value of the array
                     .withColumn(f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_array")))  # Max value of the array
                     .withColumn(f"f_mean_num_views_between_purchases_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_array")))  # Mean value of the array
                     # Compute min, max, mean for 'view_count_per_max_days_array'
                     .withColumn(f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_per_max_days_array")))  # Min of views/max_days ratio
                     .withColumn(f"f_max_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_per_max_days_array")))  # Max of views/max_days ratio
                     .withColumn(f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_per_max_days_array")))  # Mean of views/max_days ratio
                     # Fill null values with -1
                     .fillna(0, subset=[f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_mean_num_views_between_purchases_in_{cat_type}_cat",
                                         f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                                         f"f_max_views_between_purchases_per_days_in_{cat_type}_cat",
                                         f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat"])  # Fill nulls with -1
                     # Drop the arrays if no longer needed
                     .drop("view_count_array", "view_count_per_max_days_array")
          )
    print( df_res_features.count() )
    print( df_res_features.show() )
    return df_res_features


#### grouped and collect:

In [23]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.sql import Window
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.sql import Window   

def _leaf_condition(df1, df2, cat_type="LEAF"):
    if cat_type == "LEAF":
        return (df1.LEAF_CATEG_ID == df2.LEAF_CATEG_ID)
    elif cat_type == "LVL2":
        return (df1.CATEG_LVL2_ID == df2.CATEG_LVL2_ID)
    else:
        return (df1.META_CATEG_ID == df2.META_CATEG_ID)  

def _leaf_id_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_ID
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_ID
    else:
        return df.META_CATEG_ID
    
def _leaf_name_col(df, cat_type="LEAF"):
    if cat_type == "LEAF":
        return df.LEAF_CATEG_NAME
    elif cat_type == "LVL2":
        return df.CATEG_LVL2_NAME
    else:
        return df.META_CATEG_NAME

# Function to calculate the number of views between purchases and max days
def _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Step 1: Get unique purchase timestamps per buyer and category
    purchase_df = df_prch.select("BUYER_ID", _leaf_id_col(df_prch, cat_type), "EVENT_TIMESTAMP").distinct()
    
    # Step 2: Create a window to identify the next purchase timestamp for each buyer and category
    window_spec = Window.partitionBy("BUYER_ID", _leaf_id_col(df_prch, cat_type)).orderBy(F.col("EVENT_TIMESTAMP").asc())
    
    # Add the next purchase timestamp using lead
    purchase_df = purchase_df.withColumn("next_purchase_timestamp", F.lead("EVENT_TIMESTAMP", 1).over(window_spec))

    # Step 3: Join views with purchases to find views that fall between purchases
    view_purchase_joined = df_vi.join(
        purchase_df,
        (df_vi.BUYER_ID == purchase_df.BUYER_ID) & _leaf_condition(purchase_df, df_vi, cat_type), how="left"
    ).select(
        df_vi.BUYER_ID,
        df_vi.ITEM_ID,
        df_vi.EVENT_TIMESTAMP.alias("view_timestamp"),  # Rename view timestamp for clarity
        purchase_df.EVENT_TIMESTAMP.alias("purchase_timestamp"),  # Rename purchase timestamp for clarity
        _leaf_id_col(purchase_df, cat_type),  
        purchase_df.next_purchase_timestamp
    )
    

    # Step 4: Filter views to ensure each view is associated with the closest past purchase
    view_purchase_filtered = view_purchase_joined.filter(
        (view_purchase_joined.view_timestamp > view_purchase_joined.purchase_timestamp) & 
        (view_purchase_joined.view_timestamp <= F.coalesce(view_purchase_joined.next_purchase_timestamp, F.lit('9999-12-31')))
    )
    
    # Step 5: Use a window function to rank views per buyer and category and purchase -- a specific view associated to 1 purchase.
    view_ranked = view_purchase_filtered.withColumn(
        "rank",
        F.row_number().over(Window.partitionBy(view_purchase_filtered.BUYER_ID, _leaf_id_col(view_purchase_filtered, cat_type),
                                               view_purchase_filtered.view_timestamp )
                             .orderBy(F.col("purchase_timestamp").desc()))  # Rank by the closest purchase
    )
    
    # Step 6: Keep only the highest-ranked view (the closest to the purchase)
    view_single_assigned = view_ranked.filter(F.col("rank") == 1)
    
    # Step 7: Select the desired columns and add time_diff
    df_res = view_single_assigned.select(
        "BUYER_ID",
        "ITEM_ID",
        _leaf_id_col(view_single_assigned, cat_type),
        F.col("purchase_timestamp"),
        F.col("view_timestamp"),
        F.datediff(F.col("view_timestamp"), F.col("purchase_timestamp")).alias("time_diff")
    )

    # Step 8: Join with df_base and select the relevant columns
    df_res = (df_res.join(df_base, 
                          (df_res["BUYER_ID"] == df_base["BUYER_ID"]) &  
                          _leaf_condition(df_res, df_base, cat_type), 
                          how="inner")  # Use inner join to filter based on conditions
                  .filter((F.datediff(df_base.EVENT_TIMESTAMP, df_res.view_timestamp) > 0) & 
                    (F.datediff(df_base.EVENT_TIMESTAMP, df_res.purchase_timestamp) > 0)) 
              # Filter rows based on date difference
                  .select(
                      df_res.BUYER_ID,
                      _leaf_id_col(df_res, cat_type),
                      df_base.EVENT_TIMESTAMP,
                      df_res.purchase_timestamp,
                      df_res.view_timestamp,  # Select view timestamp without alias
                      df_res.time_diff.alias("days_diff")  # Alias for time difference
                  )
              )

    
    # Step 9: Count the views between purchases and calculate max days
    df_view_count = df_res.groupBy("BUYER_ID", _leaf_id_col(df_res, cat_type),"EVENT_TIMESTAMP", "purchase_timestamp") \
                          .agg(F.count(F.col("view_timestamp")).alias("view_count"),
                               F.max(F.col("days_diff")).alias("max_days_diff"))

    # Step 10: Add new feature (views / max_days_diff)

    df_view_count = df_view_count.withColumn(
    "view_count_per_max_days",
    F.when(F.col("max_days_diff") > 0, F.col("view_count") / F.col("max_days_diff")).otherwise(F.lit(0))
)
   
    
    print("step of Collect all view counts into an array per buyer and category")
 #   print( df_view_count.count() )
 #   print( df_view_count.show() )
    return df_view_count

# Function to add the features to the base dataframe
def f_num_views_from_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Get the number of views between purchases for the given category type
    df_view_counts = _num_views_from_purchase_in_cat(df_prch, df_vi, df_base, cat_type)

    # Step 2: Join df_base with df_view_counts to add the new features
    df_res = (df_base.join(df_view_counts, 
                           (df_base.BUYER_ID == df_view_counts.BUYER_ID) & 
                           (df_base.EVENT_TIMESTAMP == df_view_counts.EVENT_TIMESTAMP)& 
                           _leaf_condition(df_base, df_view_counts, cat_type), 
                           how="left").groupBy(df_base.BUYER_ID, _leaf_id_col(df_base, cat_type),df_base.EVENT_TIMESTAMP) \
                                       .agg(F.collect_list(F.col("view_count")).alias("view_count_array"),
                                            F.collect_list(F.col("view_count_per_max_days")).alias("view_count_per_max_days_array"))
                     .select(df_base.BUYER_ID, 
                             _leaf_id_col(df_base, cat_type),
                             df_base.EVENT_TIMESTAMP,
                             F.col("view_count_array"),  # Include the collected view counts
                             F.col("view_count_per_max_days_array"))  # Include the collected views per max days
             )
    
 #   print( df_res.count() )
 #   print( df_res.show() )
    # Step 3: Calculate features based on view counts (min, max, mean)
    df_res_features = (df_res.withColumn(f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_array")))  # Min value of the array
                     .withColumn(f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_array")))  # Max value of the array
                     .withColumn(f"f_mean_num_views_between_purchases_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_array")))  # Mean value of the array
                     # Compute min, max, mean for 'view_count_per_max_days_array'
                     .withColumn(f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_min(F.col("view_count_per_max_days_array")))  # Min of views/max_days ratio
                     .withColumn(f"f_max_views_between_purchases_per_days_in_{cat_type}_cat", 
                            F.array_max(F.col("view_count_per_max_days_array")))  # Max of views/max_days ratio
                     .withColumn(f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat", 
                            fe.array_mean(F.col("view_count_per_max_days_array")))  # Mean of views/max_days ratio
                     # Fill null values with -1
                     .fillna(0, subset=[f"f_min_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_max_num_views_between_purchases_in_{cat_type}_cat", 
                                         f"f_mean_num_views_between_purchases_in_{cat_type}_cat",
                                         f"f_min_views_between_purchases_per_days_in_{cat_type}_cat", 
                                         f"f_max_views_between_purchases_per_days_in_{cat_type}_cat",
                                         f"f_mean_views_between_purchases_per_days_in_{cat_type}_cat"])  # Fill nulls with -1
                     # Drop the arrays if no longer needed
                     .drop("view_count_array", "view_count_per_max_days_array")
          )
  #  print( df_res_features.count() )
  #  print( df_res_features.show() )
    return df_res_features


In [18]:
df_smp_sample.count()

5

In [16]:
df_tsf= f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

step of Collect all view counts into an array per buyer and category


3


+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP| purchase_timestamp|view_count|max_days_diff|view_count_per_max_days|
+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|1498530061|       168135|     2024-05-01|2023-09-11 02:49:50|         2|          124|   0.016129032258064516|
|1498530061|       168135|     2024-05-01|2024-01-15 21:19:59|        14|           70|                    0.2|
|1498530061|       168135|     2024-05-01|2024-03-25 03:46:10|        13|           15|     0.8666666666666667|
+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+

None


5


+----------+-------------+---------------+----------------+-----------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+----------------+-----------------------------+
|1132069035|        24410|     2024-05-01|              []|                           []|
|  11245860|       127413|     2024-05-01|              []|                           []|
|1324204268|         2885|     2024-05-01|              []|                           []|
|1498530061|       168135|     2024-05-01|     [2, 14, 13]|         [0.01612903225806...|
|  11245860|        47173|     2024-05-01|              []|                           []|
+----------+-------------+---------------+----------------+-----------------------------+

None


5


+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_between_purchases_per_days_in_LEAF_cat|f_max_views_between_purchases_per_days_in_LEAF_cat|f_mean_views_between_purchases_per_days_in_LEAF_cat|
+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------

In [17]:
df_tsf= f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")

step of Collect all view counts into an array per buyer and category


50


+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP| purchase_timestamp|view_count|max_days_diff|view_count_per_max_days|
+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|1498530061|        11700|     2024-05-01|2023-12-15 12:37:57|        37|            6|      6.166666666666667|
|1498530061|        11700|     2024-05-01|2024-04-09 04:55:59|        15|            0|                    0.0|
|1498530061|        11700|     2024-05-01|2023-12-14 04:22:02|        20|            0|                    0.0|
|1498530061|        11700|     2024-05-01|2024-04-04 05:33:33|         6|            0|                    0.0|
|1498530061|        11700|     2024-05-01|2024-01-20 05:30:47|         3|            3|                    1.0|
|1498530061|        11700|     2024-05-01|2024-02-18 18:37:17|        11|            0|                 

4


+----------+-------------+---------------+--------------------+-----------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|    view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+--------------------+-----------------------------+
|  11245860|          260|     2024-05-01|                  []|                           []|
|1324204268|        64482|     2024-05-01|                  []|                           []|
|1498530061|        11700|     2024-05-01|[2, 2, 10, 20, 11...|         [0.66666666666666...|
|1132069035|        64482|     2024-05-01|                 [3]|                       [0.25]|
+----------+-------------+---------------+--------------------+-----------------------------+

None


4


+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|f_min_num_views_between_purchases_in_META_cat|f_max_num_views_between_purchases_in_META_cat|f_mean_num_views_between_purchases_in_META_cat|f_min_views_between_purchases_per_days_in_META_cat|f_max_views_between_purchases_per_days_in_META_cat|f_mean_views_between_purchases_per_days_in_META_cat|
+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------

In [25]:
df_tsf= f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")

step of Collect all view counts into an array per buyer and category


2


+----------+-------------+---------------+--------------------+-----------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|    view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+--------------------+-----------------------------+
|1498530061|        11700|     2024-05-01|[9, 28, 24, 1, 13...|         [1.8, 5.6, 8.0, 0...|
|1132069035|        64482|     2024-05-01|                 [3]|                       [0.25]|
+----------+-------------+---------------+--------------------+-----------------------------+

None


5


+----------+-------------+---------------+--------------------+-----------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|    view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+--------------------+-----------------------------+
|  11245860|          260|     2024-05-01|                null|                         null|
|  11245860|          260|     2024-05-01|                null|                         null|
|1498530061|        11700|     2024-05-01|[14, 26, 22, 6, 1...|         [14.0, 26.0, 2.0,...|
|1324204268|        64482|     2024-05-01|                null|                         null|
|1132069035|        64482|     2024-05-01|                 [3]|                       [0.25]|
+----------+-------------+---------------+--------------------+-----------------------------+

None


5


+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|f_min_num_views_between_purchases_in_META_cat|f_max_num_views_between_purchases_in_META_cat|f_mean_num_views_between_purchases_in_META_cat|f_min_views_between_purchases_per_days_in_META_cat|f_max_views_between_purchases_per_days_in_META_cat|f_mean_views_between_purchases_per_days_in_META_cat|
+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------

In [17]:
df_tsf= f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP| purchase_timestamp|view_count|max_days_diff|view_count_per_max_days|
+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+
|1498530061|       168135|     2024-05-01|2023-09-11 02:49:50|         2|          124|   0.016129032258064516|
|1498530061|       168135|     2024-05-01|2024-01-15 21:19:59|        14|           70|                    0.2|
|1498530061|       168135|     2024-05-01|2024-03-25 03:46:10|        13|           15|     0.8666666666666667|
+----------+-------------+---------------+-------------------+----------+-------------+-----------------------+

None
step of Collect all view counts into an array per buyer and category


1


+----------+-------------+---------------+----------------+-----------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+----------------+-----------------------------+
|1498530061|       168135|     2024-05-01|     [14, 2, 13]|         [0.2, 0.016129032...|
+----------+-------------+---------------+----------------+-----------------------------+

None


5


+----------+-------------+---------------+----------------+-----------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|view_count_array|view_count_per_max_days_array|
+----------+-------------+---------------+----------------+-----------------------------+
|1132069035|        24410|     2024-05-01|            null|                         null|
|1498530061|       168135|     2024-05-01|     [2, 13, 14]|         [0.01612903225806...|
|  11245860|       127413|     2024-05-01|            null|                         null|
|  11245860|        47173|     2024-05-01|            null|                         null|
|1324204268|         2885|     2024-05-01|            null|                         null|
+----------+-------------+---------------+----------------+-----------------------------+

None


5


+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_between_purchases_per_days_in_LEAF_cat|f_max_views_between_purchases_per_days_in_LEAF_cat|f_mean_views_between_purchases_per_days_in_LEAF_cat|
+----------+-------------+---------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------

In [19]:
df_tsf.count()

5

#### Testing results of grouped feature and collect- updated feature:

In [22]:
df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

step of Collect all view counts into an array per buyer and category


+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_between_purchases_per_days_in_LEAF_cat|f_max_views_between_purchases_per_days_in_LEAF_cat|f_mean_views_between_purchases_per_days_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+---

5
step of Collect all view counts into an array per buyer and category


+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_

5
step of Collect all view counts into an array per buyer and category


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+------------------

5


In [23]:
df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_between_purchases_per_days_in_LEAF_cat|f_max_views_between_purchases_per_days_in_LEAF_cat|f_mean_views_between_purchases_per_days_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+---

5


+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_

5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+------------------

7


#### sample 1000 buyers, add features f_num_views_from_purchase_in_cat

In [ ]:
import features as F

# Step 1: Sample 20 buyers from df_smp
sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(1000)).collect()
buyer_ids = [row["BUYER_ID"] for row in sample_buyers]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those 20 buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(buyer_ids))


num_columns = len(df_smp_sample.columns)
print(f"Number of columns: {num_columns}")
# Check the number of rows
num_rows = df_smp_sample.count()
print(f"Number of rows: {num_rows}")

# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")
print(df_res_t.count())
# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")
print(df_res_t.count())
# Step 5: Calculate features for META category
df_ft_meta = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")
num_columns = len(df_res_t.columns)
print(f"Number of columns: {num_columns}")
# Check the number of rows
num_rows = df_res_t.count()
print(f"Number of rows: {num_rows}")
# Step 6: Show the results
df_res_t.show()

In [9]:
import features as F

# Step 1: Sample 20 buyers from df_smp
sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(100)).collect()
buyer_ids = [row["BUYER_ID"] for row in sample_buyers]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those 20 buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(buyer_ids))

In [12]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np

#sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(30)).collect()
# Step 1: Define the specific buyer IDs you want to sample
specific_buyer_ids = [11245860, 1498530061, 1324204268]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))


In [ ]:
# Step 1: Define the specific buyer IDs and leaf category IDs you want to sample
specific_buyer_ids = [1498530061]
specific_leaf_id = 33089

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers and leaf category
df_smp_sample = df_smp.filter((df_smp.BUYER_ID.isin(specific_buyer_ids)) & (df_smp.LEAF_CATEG_ID == specific_leaf_id))
df_prch_sample = df_prch.filter((df_prch.BUYER_ID.isin(specific_buyer_ids)) & (df_prch.LEAF_CATEG_ID == specific_leaf_id))
df_vi_sample = df_vi.filter((df_vi.BUYER_ID.isin(specific_buyer_ids)) & (df_vi.LEAF_CATEG_ID == specific_leaf_id))

# Show the filtered results for verification
df_smp_sample.show()
df_prch_sample.show()
df_vi_sample.show()


In [15]:

# Step 3: Calculate features for LEAF category
df_ft_leaf = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 6: Show the results
df_res_t.show()

step of Collect all view counts into an array per buyer and category


24/10/27 13:27:58 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_between_purchases_per_days_in_LEAF_cat|f_max_views_between_purchases_per_days_in_LEAF_cat|f_mean_views_between_purchases_per_days_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+---

In [9]:
##views per days:
df_ft_leaf = f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 6: Show the results
df_res_t.show()

step of adding vi


+----------+------------+--------------------+-------------------+-------------+-----------------------+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|next_purchase_timestamp|
+----------+------------+--------------------+-------------------+-------------+-----------------------+
|  11245860|314949668068|2024-05-22 14:23:...|2023-10-27 03:56:55|       123422|    2024-01-17 01:07:48|
|  11245860|314949668068|2024-05-22 14:23:...|2024-01-17 01:07:48|       123422|    2024-04-12 14:58:26|
|  11245860|314949668068|2024-05-22 14:23:...|2024-04-12 14:58:26|       123422|    2024-05-22 14:28:53|
|  11245860|314949668068|2024-05-22 14:23:...|2024-05-22 14:28:53|       123422|                   null|
|  11245860|113812810152|2024-05-22 14:26:...|2023-10-27 03:56:55|       123422|    2024-01-17 01:07:48|
|  11245860|113812810152|2024-05-22 14:26:...|2024-01-17 01:07:48|       123422|    2024-04-12 14:58:26|
|  11245860|113812810152|2024-05-22 14:26:...|2024-04-1

+----------+------------+--------------------+-------------------+-------------+-----------------------+----+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|next_purchase_timestamp|rank|
+----------+------------+--------------------+-------------------+-------------+-----------------------+----+
|  11245860|196381095272|2024-05-13 01:02:...|2024-05-13 00:59:06|          701|                   null|   1|
|  11245860|305608766291|2024-06-08 19:52:...|2024-05-07 07:48:09|       127424|                   null|   1|
|1324204268|203452793160|2023-10-08 17:31:...|2023-09-16 16:19:29|       261987|                   null|   1|
|1498530061|386822786231|2024-04-03 23:42:...|2023-07-31 12:36:40|       260205|                   null|   1|
|  11245860|276462762779|2024-05-15 05:00:...|2024-05-14 03:12:36|        17734|    2024-05-16 01:08:36|   1|
|  11245860|235544490440|2024-05-09 07:43:...|2024-03-04 20:40:41|        68114|                   null|   1|
|  1124586

24/10/15 11:38:17 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+-------------------+--------------------+---------+
|  BUYER_ID|LEAF_CATEG_ID| purchase_timestamp|      view_timestamp|days_diff|
+----------+-------------+-------------------+--------------------+---------+
|1498530061|       168135|2024-03-25 03:46:10|2024-04-08 21:08:...|       14|
|1498530061|       168135|2024-01-15 21:19:59|2024-03-25 03:45:...|       70|
|1498530061|       168135|2024-01-15 21:19:59|2024-03-25 03:45:...|       70|
|1498530061|       168135|2024-01-15 21:19:59|2024-02-06 02:21:...|       22|
|1498530061|       168135|2024-01-15 21:19:59|2024-02-17 18:25:...|       33|
|1498530061|       168135|2024-01-15 21:19:59|2024-02-06 01:53:...|       22|
|1498530061|       168135|2024-03-25 03:46:10|2024-04-01 02:44:...|        7|
|1498530061|       168135|2024-01-15 21:19:59|2024-02-05 21:33:...|       21|
|1498530061|       168135|2024-01-15 21:19:59|2024-02-21 00:26:...|       37|
|1498530061|       168135|2024-01-15 21:19:59|2024-03-25 03:45:.

+----------+-------------+-------------------+----------+-------------+-----------------------+
|  BUYER_ID|LEAF_CATEG_ID| purchase_timestamp|view_count|max_days_diff|view_count_per_max_days|
+----------+-------------+-------------------+----------+-------------+-----------------------+
|1498530061|       168135|2024-01-15 21:19:59|        14|           70|                    0.2|
|1498530061|       168135|2024-03-25 03:46:10|        13|           15|     0.8666666666666667|
|1498530061|       168135|2023-09-11 02:49:50|         2|          124|   0.016129032258064516|
+----------+-------------+-------------------+----------+-------------+-----------------------+

step of Collect all view counts into an array per buyer and category


24/10/15 11:40:49 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+-------------------+----------+-------------+-----------------------+
|  BUYER_ID|LEAF_CATEG_ID| purchase_timestamp|view_count|max_days_diff|view_count_per_max_days|
+----------+-------------+-------------------+----------+-------------+-----------------------+
|1498530061|       168135|2024-01-15 21:19:59|        14|           70|                    0.2|
|1498530061|       168135|2024-03-25 03:46:10|        13|           15|     0.8666666666666667|
|1498530061|       168135|2023-09-11 02:49:50|         2|          124|   0.016129032258064516|
+----------+-------------+-------------------+----------+-------------+-----------------------+



24/10/15 11:42:48 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+------------------------------------+------------------------------------+-------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_per_max_days_in_LEAF_cat|f_max_views_per_max_days_in_LEAF_cat|f_mean_views_per_max_days_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+-------------------------------

In [3]:
import features as F

# Step 1: Sample 20 buyers from df_smp
sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(3)).collect()
buyer_ids = [row["BUYER_ID"] for row in sample_buyers]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those 3 buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(buyer_ids))

# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")
df_res_t.show()

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+-----------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+------------------------------------+------------------------------------+-------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|  META_CATEG_NAME|f_min_num_views_between_purchases_in_LEAF_cat|f_max_num_views_between_purchases_in_LEAF_cat|f_mean_num_views_between_purchases_in_LEAF_cat|f_min_views_per_max_days_in_LEAF_cat|f_max_views_per_max_days_in_LEAF_cat|f_mean_views_per_max_days_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+-----------------+----------------------------------------

In [3]:
import features as F

# Step 1: Sample 20 buyers from df_smp
sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(10)).collect()
buyer_ids = [row["BUYER_ID"] for row in sample_buyers]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those 20 buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(buyer_ids))

# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = F.f_num_views_from_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")


df_res_t.show()

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+------------------------------------+------------------------------------+-------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+------------------------------------+------------------------------------+-------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+------------------------------------+------------------------------------+-------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label

## time since last purchase and num views 

In [36]:
from pyspark.sql import functions as F
from pyspark.sql import Window


# Helper method to compute views and time-related features between purchases
def _compute_view_based_features(df_prch, df_vi, df_base, cat_type="LEAF"):
    # Step 1: Join purchases with views and calculate the time difference
    df_view_join = (df_prch
                    .join(df_vi, 
                          (df_prch.BUYER_ID == df_vi.BUYER_ID) & 
                          _leaf_condition(df_prch, df_vi, cat_type), how="left")
                    .select(
                        df_vi.BUYER_ID,
                        df_vi.ITEM_ID,
                        df_vi.EVENT_TIMESTAMP.alias("view_timestamp"), 
                        df_prch.EVENT_TIMESTAMP.alias("purchase_timestamp"), 
                        _leaf_id_col(df_prch, cat_type) 
                    )
                    .withColumn("time_diff", F.datediff(F.col("view_timestamp"), F.col("purchase_timestamp")))
                    .filter(F.col("time_diff") > 0)  
                   )
   # print("join")
   # print(df_view_join.show())

    # Step 4: Join df_view_join with df_base to filter views that happen after the base event timestamp
    df_view_join = (df_view_join
                    .join(df_base, 
                          (df_view_join.BUYER_ID == df_base.BUYER_ID) & 
                          _leaf_condition(df_view_join, df_base, cat_type), how="inner")
                     .filter((F.datediff(df_base.EVENT_TIMESTAMP, F.col("view_timestamp")) > 0))
                    .select(
                        df_view_join.BUYER_ID,
                        df_view_join.ITEM_ID,
                        df_view_join.view_timestamp,
                        df_view_join.purchase_timestamp,
                        _leaf_id_col(df_view_join, cat_type),
                        df_view_join.time_diff
                           
                    )
                   )
    
 #   print("views filtered based on base timestamp")
 #   print(df_view_join.show())

    # New Step: Ensure each view is associated with the closest preceding purchase
    # Step 5: Use a window function to rank views per buyer and category
    view_ranked = df_view_join.withColumn(
        "rank", 
        F.row_number().over(Window.partitionBy(
            df_view_join.BUYER_ID,
            _leaf_id_col(df_view_join, cat_type),
            df_view_join.view_timestamp
        ).orderBy(F.desc("purchase_timestamp")))
    )
   # print(view_ranked.show())
    # Step 6: Keep only the highest-ranked view (the closest to the purchase)
    view_single_assigned = view_ranked.filter(F.col("rank") == 1).drop("rank")

    # Step 7: Assign row number to each view-purchase pair, ordering by purchase date descending per buyer have last purchase.
    window_spec = Window.partitionBy(view_single_assigned.BUYER_ID, _leaf_id_col(df_prch, cat_type) ).orderBy(F.desc("purchase_timestamp"))
    view_single_assigned = view_single_assigned.withColumn("rn", F.rank().over(window_spec))

    #print("last purchase rank")
    #view_single_assigned.show()
    
    view_single = view_single_assigned.filter(F.col("rn") == 1).drop("rn")
    
    # Step 5: Group by user, category, and purchase timestamp to calculate features
    df_grouped = (view_single.groupBy(
                        view_single.BUYER_ID,  
                        _leaf_id_col(view_single, cat_type), 
                        view_single.purchase_timestamp)
                   .agg(
                        F.count("view_timestamp").alias("num_views"),  # Count views since last purchase
                        F.max("view_timestamp").alias("last_view_timestamp"),  # Last view timestamp
                        F.max("time_diff").alias("days_since_last_view")  # Time since the last view to purchase
                    )
                 )

   # print("last view and days diff")
   # df_grouped.show()

    # Step 6: Filter rows where the last view is not null
    df_grouped = df_grouped.filter(F.col("last_view_timestamp").isNotNull())

    return df_grouped

# Feature generation function to add the number of views and days since last view
def f_views_since_last_purchase_in_cat(df_base, df_prch, df_vi, cat_type="LEAF"):
    # Step 1: Compute the view-based features
    df_view_features = _compute_view_based_features(df_prch, df_vi, df_base, cat_type)

    # Step 2: Join the computed features with the base dataset
    df_res = (df_base.join(df_view_features, 
                           (df_base.BUYER_ID == df_view_features.BUYER_ID) & 
                           (F.datediff(df_base.EVENT_TIMESTAMP, F.col("last_view_timestamp")) > 0)&
                           _leaf_condition(df_base, df_view_features, cat_type), 
                           how="left")
              .select(
                  df_base.BUYER_ID,
                  _leaf_id_col(df_base, cat_type),
                  df_base.EVENT_TIMESTAMP,
                  df_view_features.num_views.alias(f"num_views_since_last_purchase_in_{cat_type}_cat"),
                  df_view_features.days_since_last_view.alias(f"time_from_last_view_to_last_purchase_in_{cat_type}_cat")))
     

    #print("grouped joined result")
    #df_res.show()
    df_res = df_res.fillna(0, subset=[f"num_views_since_last_purchase_in_{cat_type}_cat"])
    df_res = df_res.fillna(-1, subset=[f"time_from_last_view_to_last_purchase_in_{cat_type}_cat"])

    #print("final joined result")
    #print(df_res.show())
    df_res_no_dups = df_res.distinct()

    return df_res_no_dups 

In [34]:
df_tsf= f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

final joined result


+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  11245860|        47173|     2024-05-01|                                        0|                                              -1|
|1132069035|        24410|     2024-05-01|                                        0|                                              -1|
|  11245860|       127413|     2024-05-01|                                        0|                                              -1|
|1498530061|       168135|     2024-05-01|                                       12|                                              15|
|1324204268|         2885|     2024-05-01|                    

In [27]:
df_tsf= f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")

join


+----------+------------+--------------------+-------------------+-------------+---------+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff|
+----------+------------+--------------------+-------------------+-------------+---------+
|  11245860|314949668068|2024-05-22 14:23:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|113812810152|2024-05-22 14:26:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|392953153725|2024-05-22 14:25:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|143698575191|2024-05-22 14:28:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|314949668068|2024-05-22 14:23:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|113812810152|2024-05-22 14:26:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|392953153725|2024-05-22 14:25:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|143698575191|2024-05-22 14:28:...|2024-04-12 14:58:26|       123422|       40|

+----------+------------+--------------------+-------------------+-------------+---------+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff|
+----------+------------+--------------------+-------------------+-------------+---------+
|1498530061|295495274519|2024-02-12 05:33:...|2024-01-15 21:19:59|       168135|       28|
|1498530061|386886812375|2024-04-09 06:36:...|2024-01-15 21:19:59|       168135|       85|
|1498530061|386886812375|2024-04-09 06:36:...|2024-01-15 21:19:59|       168135|       85|
|1498530061|386886812375|2024-04-08 21:08:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|386886812375|2024-04-08 21:08:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|386886812375|2024-04-08 21:07:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|335284547649|2024-04-08 22:02:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|335284547649|2024-04-08 22:02:...|2024-01-15 21:19:59|       168135|       84|

+----------+------------+--------------------+-------------------+-------------+---------+----+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff|rank|
+----------+------------+--------------------+-------------------+-------------+---------+----+
|1498530061|386886812375|2024-04-08 21:08:...|2024-03-25 03:46:10|       168135|       14|   1|
|1498530061|386886812375|2024-04-08 21:08:...|2024-01-15 21:19:59|       168135|       84|   2|
|1498530061|386886812375|2024-04-08 21:08:...|2023-09-11 02:49:50|       168135|      210|   3|
|1498530061|386886812375|2024-04-08 21:08:...|2023-08-02 04:23:18|       168135|      250|   4|
|1498530061|176279071107|2024-03-25 03:45:...|2024-01-15 21:19:59|       168135|       70|   1|
|1498530061|176279071107|2024-03-25 03:45:...|2023-09-11 02:49:50|       168135|      196|   2|
|1498530061|176279071107|2024-03-25 03:45:...|2023-08-02 04:23:18|       168135|      236|   3|
|1498530061|176279071107|2024-03-25 03:4

+----------+------------+--------------------+-------------------+-------------+---------+---+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff| rn|
+----------+------------+--------------------+-------------------+-------------+---------+---+
|1498530061|335284547649|2024-04-01 02:44:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|386886812375|2024-04-08 21:07:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|386886812375|2024-04-08 21:08:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|335284547649|2024-04-08 22:02:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|386886812375|2024-04-09 06:36:...|2024-03-25 03:46:10|       168135|       15|  1|
|1498530061|335284547649|2024-04-01 02:44:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|335284547649|2024-04-01 02:47:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|386886812375|2024-04-08 21:08:...|2024

+----------+-------------+-------------------+---------+--------------------+--------------------+
|  BUYER_ID|LEAF_CATEG_ID| purchase_timestamp|num_views| last_view_timestamp|days_since_last_view|
+----------+-------------+-------------------+---------+--------------------+--------------------+
|1498530061|       168135|2024-03-25 03:46:10|       12|2024-04-09 06:36:...|                  15|
+----------+-------------+-------------------+---------+--------------------+--------------------+

final joined result


+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|1498530061|       168135|     2024-05-01|                                       12|                                              15|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+

final joined result


+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|1498530061|       168135|     2024-05-01|                                       12|                                              15|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+

None


#### calculating features based on sample per caeg level-- total 6

In [38]:
df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.count())

df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

5


+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|num_views_since_last_purchase_in_LVL2_cat|time_from_last_view_to_last_purchase_in_LVL2_cat|
+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------

5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|num_views_since_last_purchase_in_LVL2_cat|time_from_last_view_to_last_purchase_in_LVL2_cat|num_views_since_last_purchase_in_META_cat|time_from_last_view_to_last_purchase_in_META_cat|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------

5


In [37]:
df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+-----------------------------------------+------------------------------------------------+
|1132069035|        24410|     2024-05-01|              0|    0|Sports Mem, Cards...|        24409|Fan Apparel & Sou...|        64482|Sports Mem, Cards...|                                        0|                                              -1|
|1498530061|

5


5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|num_views_since_last_purchase_in_LVL2_cat|time_from_last_view_to_last_purchase_in_LVL2_cat|num_views_since_last_purchase_in_META_cat|time_from_last_view_to_last_purchase_in_META_cat|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------

5


In [4]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np

#sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(30)).collect()
# Step 1: Define the specific buyer IDs you want to sample
specific_buyer_ids = [11245860, 1498530061, 1324204268]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))


In [5]:
import features as F


# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")


df_res_t.show()

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+---------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|num_views_since_last_purchase_in_LVL2_cat|time_from_last_view_to_last_purchase_in_LVL2_cat|num_views_since_last_purchase_in_META_cat|time_from_last_view_to_last_purchase_in_META_cat|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+---------------+--

In [40]:
df_ft_leaf = views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

join


+----------+------------+--------------------+-------------------+-------------+---------+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff|
+----------+------------+--------------------+-------------------+-------------+---------+
|  11245860|314949668068|2024-05-22 14:23:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|113812810152|2024-05-22 14:26:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|392953153725|2024-05-22 14:25:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|143698575191|2024-05-22 14:28:...|2024-01-17 01:07:48|       123422|      126|
|  11245860|314949668068|2024-05-22 14:23:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|113812810152|2024-05-22 14:26:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|392953153725|2024-05-22 14:25:...|2024-04-12 14:58:26|       123422|       40|
|  11245860|143698575191|2024-05-22 14:28:...|2024-04-12 14:58:26|       123422|       40|

24/10/20 09:45:53 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+------------+--------------------+-------------------+-------------+---------+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff|
+----------+------------+--------------------+-------------------+-------------+---------+
|1498530061|295495274519|2024-02-12 05:33:...|2024-01-15 21:19:59|       168135|       28|
|1498530061|386886812375|2024-04-09 06:36:...|2024-01-15 21:19:59|       168135|       85|
|1498530061|386886812375|2024-04-09 06:36:...|2024-01-15 21:19:59|       168135|       85|
|1498530061|386886812375|2024-04-08 21:08:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|386886812375|2024-04-08 21:08:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|386886812375|2024-04-08 21:07:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|335284547649|2024-04-08 22:02:...|2024-01-15 21:19:59|       168135|       84|
|1498530061|335284547649|2024-04-08 22:02:...|2024-01-15 21:19:59|       168135|       84|

24/10/20 09:47:02 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

...

24/10/20 09:48:35 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+------------+--------------------+-------------------+-------------+---------+---+
|  BUYER_ID|     ITEM_ID|      view_timestamp| purchase_timestamp|LEAF_CATEG_ID|time_diff| rn|
+----------+------------+--------------------+-------------------+-------------+---------+---+
|1498530061|386886812375|2024-04-09 06:36:...|2024-03-25 03:46:10|       168135|       15|  1|
|1498530061|335284547649|2024-04-01 02:44:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|335284547649|2024-04-08 22:02:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|335284547649|2024-04-01 02:44:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|335284547649|2024-04-08 22:02:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|386886812375|2024-04-08 21:07:...|2024-03-25 03:46:10|       168135|       14|  1|
|1498530061|335284547649|2024-04-01 02:47:...|2024-03-25 03:46:10|       168135|        7|  1|
|1498530061|386911570003|2024-04-08 21:08:...|2024

+----------+-------------+-------------------+---------+--------------------+--------------------+
|  BUYER_ID|LEAF_CATEG_ID| purchase_timestamp|num_views| last_view_timestamp|days_since_last_view|
+----------+-------------+-------------------+---------+--------------------+--------------------+
|1498530061|       168135|2024-03-25 03:46:10|       12|2024-04-09 06:36:...|                  15|
+----------+-------------+-------------------+---------+--------------------+--------------------+

final joined result


24/10/20 09:50:36 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|1498530061|       168135|     2024-05-01|                                       12|                                              15|
+----------+-------------+---------------+-----------------------------------------+------------------------------------------------+



In [38]:
df_res_t.show()

24/10/20 09:41:54 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+---------------+-----------------------------------------+------------------------------------------------+
|1498530061|       168135|     2024-05-01|              6|    1|Home & Garden:Too...|          631|Tools & Workshop ...|        11700|  Home & Garden|                                       12|                                              15|
+----------+-------------+------

In [9]:
# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")


df_res_t.show()

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+---------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_NAME|num_views_since_last_purchase_in_LEAF_cat|time_from_last_view_to_last_purchase_in_LEAF_cat|num_views_since_last_purchase_in_LVL2_cat|time_from_last_view_to_last_purchase_in_LVL2_cat|num_views_since_last_purchase_in_META_cat|time_from_last_view_to_last_purchase_in_META_cat|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+---------------+--

In [10]:
df_res_t.columns

['BUYER_ID',
 'META_CATEG_ID',
 'EVENT_TIMESTAMP',
 'CATEG_LVL2_ID',
 'LEAF_CATEG_ID',
 'num_repurchases',
 'label',
 'LEAF_CATEG_NAME',
 'CATEG_LVL2_NAME',
 'META_CATEG_NAME',
 'num_views_since_last_purchase_in_LEAF_cat',
 'time_from_last_view_to_last_purchase_in_LEAF_cat',
 'num_views_since_last_purchase_in_LVL2_cat',
 'time_from_last_view_to_last_purchase_in_LVL2_cat',
 'num_views_since_last_purchase_in_META_cat',
 'time_from_last_view_to_last_purchase_in_META_cat']

## Category entropy: 

In [52]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import math

# Function to compute entropy
def entropy(proportions):
    if proportions is None or len(proportions) == 0:
        return -1  
    entropy_value = -sum([p * math.log2(p) for p in proportions if p > 0])
    return round(abs(entropy_value), 4)

# Register the UDF for entropy calculation
entropy_udf = F.udf(entropy, "double")

# Function to calculate category diversity entropy
def add_category_entropy(df_base, df_action, action_type="purchase", cat_type="LEAF"):
    df_action_base = (
        df_base
        .join(df_action, 
              (df_base.BUYER_ID == df_action.BUYER_ID) & 
              _leaf_condition(df_base, df_action, cat_type), 
              how="left")
        .filter(F.datediff(df_base.EVENT_TIMESTAMP, df_action.EVENT_TIMESTAMP) > 0)
        .select(
            df_action.BUYER_ID,
            df_action.ITEM_ID, 
            df_action.EVENT_TIMESTAMP, 
            _leaf_id_col(df_action, cat_type))) 
   # print(" action- base") 
   # df_action_base.show()
    # Step 3: Count number of actions (purchases/views) per user per category
    df_counts = (
        df_action_base
        .groupBy(df_action_base.BUYER_ID, _leaf_id_col(df_action, cat_type))
        .agg(F.count("*").alias("cat_action_count"))
    )
    
    #print(" df_counts") 
    #df_counts.show()
    # Step 4: Calculate total number of actions per user
    df_total = (
        df_action_base
        .groupBy(df_action_base.BUYER_ID)
        .agg(F.count("*").alias("total_action_count"))
    )

    #print(" df_total") 
    #df_total.show()
    # Step 5: Join category counts with total counts
    df_joined = (
        df_counts
        .join(df_total, on="BUYER_ID", how="inner")
        .withColumn("proportion",
            F.when(F.col("total_action_count") > 0, F.col("cat_action_count") / F.col("total_action_count")).otherwise(-1)
        )
    )
    
    #print(" df_joined") 
    #df_joined.show() 
    
    # Step 6: Group by user and calculate entropy for each user
    df_entropy = (
        df_joined
        .groupBy("BUYER_ID")
        .agg(F.collect_list("proportion").alias("proportions"))
    )
  
    #print(" df_entropy proportion") 
    #df_entropy.show() 
    # Step 7: Apply entropy function to the list of proportions
    df_entropy = (
        df_entropy
        .withColumn("category_entropy", entropy_udf(F.col("proportions")))
        .select("BUYER_ID", "category_entropy")
    )
    
    #print(" df_entropy") 
    #df_entropy.show()   
    return df_entropy 

def f_category_entropy(df_base, df_action, action_type="purchase", cat_type="LEAF"):
    # Calculate category entropy for the given category type and action type
    df_entropy = add_category_entropy(df_base, df_action, action_type=action_type, cat_type=cat_type)
    
    df_res = (
        df_base
        .join(df_entropy, on="BUYER_ID", how="left")
        .withColumnRenamed("category_entropy", f"f_entropy_for_{cat_type}_cat_{action_type}s")
        .select(
            df_base.BUYER_ID,
            _leaf_id_col(df_base, cat_type),
            df_base.EVENT_TIMESTAMP,
            F.col(f"f_entropy_for_{cat_type}_cat_{action_type}s"))
    )
    df_res = df_res.fillna(-1, subset=[f"f_entropy_for_{cat_type}_cat_{action_type}s"])
    
    df_res_no_dups = df_res.distinct() 
    
    #print("final result")
    #print(df_res.show())
    return df_res_no_dups


In [47]:
df_tsf= f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="LEAF")

final result


+----------+-------------+---------------+--------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_entropy_for_LEAF_cat_purchases|
+----------+-------------+---------------+--------------------------------+
|  11245860|        47173|     2024-05-01|                          0.8113|
|  11245860|       127413|     2024-05-01|                          0.8113|
|1132069035|        24410|     2024-05-01|                             0.0|
|1324204268|         2885|     2024-05-01|                             0.0|
|1498530061|       168135|     2024-05-01|                             0.0|
+----------+-------------+---------------+--------------------------------+

None


In [54]:
df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")


df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")


df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_entropy_for_LEAF_cat_purchases|f_entropy_for_LVL2_cat_purchases|f_entropy_for_META_cat_purchases|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  11245860|          260|     2024-05-01|       181423|       127413|              0|    0|Stamps:Africa:Sou...|              Africa|              Stamps|                          0.8113|                          0.7496|              

5


In [53]:
df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="LEAF")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.count())

df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="LVL2")
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

df_ft = f_category_entropy(df_smp_sample, df_prch_sample, action_type="purchase", cat_type="META")
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count())

5


+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+--------------------------------+--------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_entropy_for_LEAF_cat_purchases|f_entropy_for_LVL2_cat_purchases|
+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+--------------------------------+--------------------------------+
|1324204268|        24409|     2024-05-01|         2885|              0|    0|Sports Mem, Cards...|Fan Apparel & Sou...|        64482|Sports Mem, Cards...|                             0.0|                             0.0|
|1132069035|        24409|     2024-05-01|        24410|              0|    0|Sports Mem, Cards...|Fan Apparel &

5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_entropy_for_LEAF_cat_purchases|f_entropy_for_LVL2_cat_purchases|f_entropy_for_META_cat_purchases|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  11245860|          260|     2024-05-01|       181423|       127413|              0|    0|Stamps:Africa:Sou...|              Africa|              Stamps|                          0.8113|                          0.7496|              

5


In [3]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np

#sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(30)).collect()
# Step 1: Define the specific buyer IDs you want to sample
specific_buyer_ids = [11245860, 1498530061, 1324204268]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))

In [10]:
# Step 3: Calculate features for LEAF category
df_ft_leaf = f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="LEAF")
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="LVL2")
df_res_t = add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="META")
df_res_t = add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")

df_res_t.show()

 action- base


+----------+------------+-------------------+-------------+
|  BUYER_ID|     ITEM_ID|    EVENT_TIMESTAMP|LEAF_CATEG_ID|
+----------+------------+-------------------+-------------+
|1498530061|404678532104|2024-01-15 21:19:59|       168135|
|1498530061|314821705039|2023-09-11 02:49:50|       168135|
|1498530061|385829607761|2023-08-02 04:23:18|       168135|
|1498530061|305469856828|2024-03-25 03:46:10|       168135|
|  11245860|305238529560|2023-11-06 20:54:19|        47173|
|  11245860|305403756032|2024-02-25 20:26:12|        47173|
|  11245860|404455166831|2023-09-03 18:39:10|        47173|
|1324204268|175992311998|2023-12-24 22:29:05|         2885|
|  11245860|305338864187|2024-01-10 21:33:54|       127413|
+----------+------------+-------------------+-------------+

 df_counts


+----------+-------------+----------------+
|  BUYER_ID|LEAF_CATEG_ID|cat_action_count|
+----------+-------------+----------------+
|  11245860|        47173|               3|
|  11245860|       127413|               1|
|1498530061|       168135|               4|
|1324204268|         2885|               1|
+----------+-------------+----------------+

 df_total


24/10/22 08:12:21 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+------------------+
|  BUYER_ID|total_action_count|
+----------+------------------+
|1498530061|                 4|
|  11245860|                 4|
|1324204268|                 1|
+----------+------------------+

 df_joined


...

+----------+------------------+
|  BUYER_ID|total_action_count|
+----------+------------------+
|  11245860|              1424|
|1498530061|               121|
|1324204268|                 1|
+----------+------------------+

 df_joined


+----------+-------------+----------------+------------------+----------+
|  BUYER_ID|META_CATEG_ID|cat_action_count|total_action_count|proportion|
+----------+-------------+----------------+------------------+----------+
|  11245860|          260|            1424|              1424|       1.0|
|1324204268|        64482|               1|                 1|       1.0|
|1498530061|        11700|             121|               121|       1.0|
+----------+-------------+----------------+------------------+----------+

 df_entropy


+----------+-----------+
|  BUYER_ID|proportions|
+----------+-----------+
|  11245860|      [1.0]|
|1324204268|      [1.0]|
|1498530061|      [1.0]|
+----------+-----------+

 df_entropy


+----------+----------------+
|  BUYER_ID|category_entropy|
+----------+----------------+
|  11245860|             0.0|
|1324204268|             0.0|
|1498530061|             0.0|
+----------+----------------+



+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+---------------------------+---------------------------+---------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_LEAF_cat_purchase_entropy|f_LVL2_cat_purchase_entropy|f_META_cat_purchase_entropy|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+---------------------------+---------------------------+---------------------------+
|  11245860|          260|     2024-05-01|       181417|        47173|              0|    0|Stamps:Latin Amer...|       Latin America|              Stamps|                     0.8113|                     0.7496|                        0.0|
|  11245860|          260|     2024-05-0

In [13]:
df_res_t.columns

['BUYER_ID',
 'META_CATEG_ID',
 'EVENT_TIMESTAMP',
 'CATEG_LVL2_ID',
 'LEAF_CATEG_ID',
 'num_repurchases',
 'label',
 'LEAF_CATEG_NAME',
 'CATEG_LVL2_NAME',
 'META_CATEG_NAME',
 'f_LEAF_cat_purchase_entropy',
 'f_LVL2_cat_purchase_entropy',
 'f_META_cat_purchase_entropy']

##### test add features calc:

In [9]:
import features as F

# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = F.f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")
print(df_res_t.count())
print(df_res_t.show())

5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_entropy_for_LEAF_cat_purchases|f_entropy_for_LVL2_cat_purchases|f_entropy_for_META_cat_purchases|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+--------------------------------+--------------------------------+--------------------------------+
|  11245860|          260|     2024-05-01|       181417|        47173|              0|    0|Stamps:Latin Amer...|       Latin America|              Stamps|                          0.8113|                          0.7496|              

In [ ]:
df_res_t.columns

In [11]:

df_res_t.select("BUYER_ID" ,"LEAF_CATEG_ID","f_entropy_for_LEAF_cat_purchases","f_entropy_for_LVL2_cat_purchases","f_entropy_for_META_cat_purchases").show()

24/10/29 11:36:20 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+--------------------------------+--------------------------------+--------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|f_entropy_for_LEAF_cat_purchases|f_entropy_for_LVL2_cat_purchases|f_entropy_for_META_cat_purchases|
+----------+-------------+--------------------------------+--------------------------------+--------------------------------+
|  11245860|        47173|                          0.8113|                          0.7496|                             0.0|
|  11245860|       127413|                          0.8113|                          0.7496|                             0.0|
|1498530061|       168135|                             0.0|                             0.0|                             0.0|
|1324204268|         2885|                             0.0|                             0.0|                             0.0|
|1132069035|        24410|                             0.0|                             0.0|                          

In [8]:
df_ft_leaf = f_category_entropy(df_smp_sample, df_prch_sample,  action_type="purchase", cat_type="LEAF")
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

 action- base


+----------+------------+-------------------+-------------+
|  BUYER_ID|     ITEM_ID|    EVENT_TIMESTAMP|LEAF_CATEG_ID|
+----------+------------+-------------------+-------------+
|1498530061|404678532104|2024-01-15 21:19:59|       168135|
|1498530061|314821705039|2023-09-11 02:49:50|       168135|
|1498530061|385829607761|2023-08-02 04:23:18|       168135|
|1498530061|305469856828|2024-03-25 03:46:10|       168135|
|  11245860|305238529560|2023-11-06 20:54:19|        47173|
|  11245860|305403756032|2024-02-25 20:26:12|        47173|
|  11245860|404455166831|2023-09-03 18:39:10|        47173|
|1324204268|175992311998|2023-12-24 22:29:05|         2885|
|  11245860|305338864187|2024-01-10 21:33:54|       127413|
+----------+------------+-------------------+-------------+

 df_counts


+----------+-------------+----------------+
|  BUYER_ID|LEAF_CATEG_ID|cat_action_count|
+----------+-------------+----------------+
|  11245860|        47173|               3|
|  11245860|       127413|               1|
|1498530061|       168135|               4|
|1324204268|         2885|               1|
+----------+-------------+----------------+

 df_total


+----------+------------------+
|  BUYER_ID|total_action_count|
+----------+------------------+
|  11245860|                 4|
|1324204268|                 1|
|1498530061|                 4|
+----------+------------------+

 df_joined


+----------+-------------+----------------+------------------+----------+
|  BUYER_ID|LEAF_CATEG_ID|cat_action_count|total_action_count|proportion|
+----------+-------------+----------------+------------------+----------+
|  11245860|       127413|               1|                 4|      0.25|
|  11245860|        47173|               3|                 4|      0.75|
|1324204268|         2885|               1|                 1|       1.0|
|1498530061|       168135|               4|                 4|       1.0|
+----------+-------------+----------------+------------------+----------+

 df_entropy


+----------+------------+
|  BUYER_ID| proportions|
+----------+------------+
|  11245860|[0.75, 0.25]|
|1324204268|       [1.0]|
|1498530061|       [1.0]|
+----------+------------+

 df_entropy


+----------+------------------+
|  BUYER_ID|  category_entropy|
+----------+------------------+
|  11245860|0.8112781244591328|
|1324204268|              -0.0|
|1498530061|              -0.0|
+----------+------------------+



In [9]:
df_res_t.show()

24/10/21 09:38:38 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_LEAF_cat_purchase_entropy|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+---------------------------+
|1498530061|       168135|     2024-05-01|              6|    1|Home & Garden:Too...|          631|Tools & Wo

In [15]:

df_res_t.select("BUYER_ID" ,"LEAF_CATEG_ID","f_LEAF_cat_purchase_entropy").show()

+----------+-------------+---------------------------+
|  BUYER_ID|LEAF_CATEG_ID|f_LEAF_cat_purchase_entropy|
+----------+-------------+---------------------------+
|  11245860|       127413|         0.8112781244591328|
|  11245860|        47173|         0.8112781244591328|
|1324204268|         2885|                       -0.0|
|1498530061|       168135|                       -0.0|
+----------+-------------+---------------------------+



## difference between mean purchases/views 

In [20]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import math
 

def f_user_vs_mean_cat_actions_in_last_x_days(df_base, df_action, cat_type="LEAF", days=30, action_type="purchase"):
    feature_name = f"f_user_vs_mean_{action_type}_from_{cat_type}_cat_in_last_{days}_days"
    
    # Step 1: Calculate user activity in the last 'days' period
    df_user_actions = (
        df_base.join(df_action, 
            (df_base.BUYER_ID == df_action.BUYER_ID) & 
            _leaf_condition(df_base, df_action, cat_type) &
            (F.datediff(df_base.EVENT_TIMESTAMP, df_action.EVENT_TIMESTAMP) > 0) &
            (F.datediff(df_base.EVENT_TIMESTAMP, df_action.EVENT_TIMESTAMP) <= days), 
            how="left")
        .groupBy(df_base.BUYER_ID, _leaf_id_col(df_base, cat_type), df_base.EVENT_TIMESTAMP)
        .agg(F.count(df_action.EVENT_TIMESTAMP).alias(f"user_{action_type}_count"))
    )

    #print (" df_user_actions" ) 
    #df_user_actions.show() 
    
    # Step 2: Calculate population mean and stddev only for values > 0
    df_population_stats = (
       # df_user_actions.filter(F.col(f"user_{action_type}_count") > 0)
        df_user_actions.groupBy(_leaf_id_col(df_user_actions, cat_type))
        .agg(
            F.mean(f"user_{action_type}_count").alias("population_mean"),
            F.stddev(f"user_{action_type}_count").alias("population_stddev")
        )
    )
    #print (" df_population_stats" ) 
    #df_population_stats.show() 
    
    # Step 3: Join user actions with population stats and calculate z-score
    df_result = (
        df_user_actions.join(
            df_population_stats, 
            on=_leaf_condition(df_user_actions, df_population_stats, cat_type), 
            how="left"
        )
        .withColumn(
            feature_name, 
            F.when(
                (F.col("population_stddev") > 0) & (F.col("population_stddev").isNotNull()), 
                (F.col(f"user_{action_type}_count") - F.col("population_mean")) / F.col("population_stddev")
            ).otherwise(0)
        )
        .fillna(0,subset=[feature_name] )
    ).select(
            df_user_actions.BUYER_ID,
            _leaf_id_col(df_user_actions, cat_type),
            df_user_actions.EVENT_TIMESTAMP,
            F.col(feature_name))
    
    df_res = df_result.distinct() 
    
    #print("df_res_no_dups")
    #print(df_res.show())
    return df_res
    
   

#### tests:

In [18]:
df_tsf= f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=30 ,action_type="purchase")


df_res_no_dups


+----------+-------------+---------------+-----------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_user_vs_mean_purchase_from_LEAF_cat_in_last_30_days|
+----------+-------------+---------------+-----------------------------------------------------+
|1324204268|         2885|     2024-05-01|                                                  0.0|
|  11245860|       127413|     2024-05-01|                                                  0.0|
|1498530061|       168135|     2024-05-01|                                                  0.0|
|  11245860|        47173|     2024-05-01|                                                  0.0|
|1132069035|        24410|     2024-05-01|                                                  0.0|
+----------+-------------+---------------+-----------------------------------------------------+

None


In [15]:
df_tsf= f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=30 ,action_type="purchase")


 df_user_actions


+----------+-------------+---------------+-------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|user_purchase_count|
+----------+-------------+---------------+-------------------+
|  11245860|        47173|     2024-05-01|                  0|
|1132069035|        24410|     2024-05-01|                  0|
|  11245860|       127413|     2024-05-01|                  0|
|1498530061|       168135|     2024-05-01|                  0|
|1324204268|         2885|     2024-05-01|                  0|
+----------+-------------+---------------+-------------------+

 df_population_stats


24/10/29 12:04:40 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+-------------+---------------+-----------------+
|LEAF_CATEG_ID|population_mean|population_stddev|
+-------------+---------------+-----------------+
|         2885|            0.0|              NaN|
|       127413|            0.0|              NaN|
|       168135|            0.0|              NaN|
|        47173|            0.0|              NaN|
|        24410|            0.0|              NaN|
+-------------+---------------+-----------------+

df_res_no_dups


NameError: name 'df_res_no_dups' is not defined

In [21]:
df_ft = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF", days=30 ,action_type="purchase")
df_res = add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="inner")
print(df_res.count())

df_ft = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LVL2", days=30 ,action_type="purchase") 
df_res = add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="inner")
print(df_res.count()) 

df_ft = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="META", days=30 ,action_type="purchase") 
df_res = add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="inner")
print(df_res.show())
print(df_res.count()) 

5


5


+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_user_vs_mean_purchase_from_LEAF_cat_in_last_30_days|f_user_vs_mean_purchase_from_LVL2_cat_in_last_30_days|f_user_vs_mean_purchase_from_META_cat_in_last_30_days|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+
|  11245860|          260|     2024-05-01|    

5


In [ ]:
import features as F


# Step 3: Calculate features for LEAF category
df_ft_leaf = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LEAF")
df_res_t = F.add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="LVL2")
df_res_t = F.add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = F.f_views_since_last_purchase_in_cat(df_smp_sample, df_prch_sample, df_vi_sample, cat_type="META")
df_res_t = F.add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")


df_res_t.show()

In [4]:
# Step 1: Sample 20 distinct buyer IDs from df_smp
sample_buyers = [row["BUYER_ID"] for row in df_smp.select("BUYER_ID").distinct().limit(100).collect()]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(sample_buyers))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(sample_buyers))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(sample_buyers))

In [4]:
from pyspark.sql import functions as F
import feature_helpers as fe
from pyspark.sql.types import FloatType
import numpy as np

#sample_buyers = (df_smp.select("BUYER_ID").distinct().limit(20)).collect()
# Step 1: Define the specific buyer IDs you want to sample
specific_buyer_ids = [11245860, 1498530061, 1324204268,20696,258212,24541,23782,15709]

# Step 2: Filter df_smp, df_prch, and df_vi to only include those specific buyers
df_smp_sample = df_smp.filter(df_smp.BUYER_ID.isin(specific_buyer_ids))
df_prch_sample = df_prch.filter(df_prch.BUYER_ID.isin(specific_buyer_ids))
df_vi_sample = df_vi.filter(df_vi.BUYER_ID.isin(specific_buyer_ids))

In [5]:
# Step 3: Calculate features for LEAF category
df_ft_leaf = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample ,cat_type="LEAF",days=90, action_type="purchase" )
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")
df_res_t.show()

 df_user_actions


+----------+-------------+---------------+-------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|user_purchase_count|
+----------+-------------+---------------+-------------------+
|    839622|        20696|     2024-05-01|                  2|
|   8248926|        23782|     2024-05-01|                  0|
|  11245860|        47173|     2024-05-01|                  1|
|  22447157|       261186|     2024-05-01|                  0|
| 102058654|       178893|     2024-05-01|                  0|
| 147993967|        29223|     2024-05-01|                  0|
| 209859385|       105516|     2024-05-01|                  0|
| 238279677|       117400|     2024-05-01|                  0|
| 326936611|          206|     2024-05-01|                  0|
| 556681734|        15053|     2024-05-01|                  0|
| 993558660|        20349|     2024-05-01|                  1|
|1033959804|       261186|     2024-05-01|                  1|
|1036833091|       121848|     2024-05-01|             

24/10/27 11:45:32 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

+-------------+---------------+------------------+
|LEAF_CATEG_ID|population_mean| population_stddev|
+-------------+---------------+------------------+
|       261044|            0.0|               NaN|
|       261999|            0.0|               NaN|
|       117400|            0.0|               NaN|
|        52365|            0.0|               0.0|
|       178893|            0.0|               0.0|
|        74925|            1.0|               NaN|
|         2885|            0.0|               NaN|
|         3226|            0.0|               NaN|
|       261028|            1.0|               NaN|
|         9355|            1.0|               NaN|
|        10960|            1.0|               NaN|
|       139806|            0.0|               NaN|
|        20349|            1.0|               NaN|
|       261624|            1.0|               NaN|
|       259859|            0.0|               NaN|
|       260115|            0.0|               NaN|
|        48706|            0.0|

+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+-----------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|num_repurchases|label|     LEAF_CATEG_NAME|CATEG_LVL2_ID|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_user_vs_mean_purchase_from_LEAF_cat_in_last_90_days|
+----------+-------------+---------------+---------------+-----+--------------------+-------------+--------------------+-------------+--------------------+-----------------------------------------------------+
|  17515348|        36032|     2024-05-01|              1|    1|Home & Garden:Kit...|        20625|Kitchen, Dining &...|        11700|       Home & Garden|                                                 -1.0|
| 727415409|        85917|     2024-05-01|              0|    0|Home & Garden:Too...|          631|Tools & Workshop ...|        11700|       Home & Garden|     

In [7]:
df_res_t.select( ["BUYER_ID", "LEAF_CATEG_ID","f_user_vs_mean_purchase_from_LEAF_cat_in_last_90_days"]).show()

+----------+-------------+-----------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|f_user_vs_mean_purchase_from_LEAF_cat_in_last_90_days|
+----------+-------------+-----------------------------------------------------+
|  67114836|       261044|                                                 -1.0|
|  53132737|       261999|                                                 -1.0|
| 238279677|       117400|                                                 -1.0|
| 782596511|        52365|                                                 -1.0|
|2592723567|        52365|                                                 -1.0|
|1122553165|       178893|                                                 -1.0|
| 102058654|       178893|                                                 -1.0|
|  80471728|        74925|                                                 -1.0|
|1324204268|         2885|                                                 -1.0|
| 113704205|         3226|  

In [8]:
# Step 3: Calculate features for LEAF category
df_ft_leaf = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample ,cat_type="LEAF",days=90, action_type="purchase" )
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

df_ft_leaf = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample ,cat_type="LEAF",days=180, action_type="purchase" )
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")

df_ft_leaf = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample ,cat_type="LEAF",days=360, action_type="purchase" )
df_res_t = add_feature(df_smp_sample, df_ft_leaf, join_key="LEAF_CATEG_ID")
# Step 4: Calculate features for LVL2 category
df_ft_lvl2 = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample ,cat_type="LVL2",days=180, action_type="purchase" )
df_res_t = add_feature(df_res_t, df_ft_lvl2, join_key="CATEG_LVL2_ID")

# Step 5: Calculate features for META category
df_ft_meta = f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="META", days=30, action_type="purchase" )
df_res_t = add_feature(df_res_t, df_ft_meta, join_key="META_CATEG_ID")

df_res_t.show()

 df_user_actions


+----------+-------------+---------------+-------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|user_purchase_count|
+----------+-------------+---------------+-------------------+
|    839622|        20696|     2024-05-01|                  2|
|   8248926|        23782|     2024-05-01|                  0|
|  11245860|        47173|     2024-05-01|                  1|
|  22447157|       261186|     2024-05-01|                  0|
| 102058654|       178893|     2024-05-01|                  0|
| 147993967|        29223|     2024-05-01|                  0|
| 209859385|       105516|     2024-05-01|                  0|
| 238279677|       117400|     2024-05-01|                  0|
| 326936611|          206|     2024-05-01|                  0|
| 556681734|        15053|     2024-05-01|                  0|
| 993558660|        20349|     2024-05-01|                  1|
|1033959804|       261186|     2024-05-01|                  1|
|1036833091|       121848|     2024-05-01|             

+-------------+---------------+------------------+
|LEAF_CATEG_ID|population_mean| population_stddev|
+-------------+---------------+------------------+
|       261044|            0.0|               NaN|
|       261999|            0.0|               NaN|
|       117400|            0.0|               NaN|
|        52365|            0.0|               0.0|
|       178893|            0.0|               0.0|
|        74925|            1.0|               NaN|
|         2885|            0.0|               NaN|
|         3226|            0.0|               NaN|
|       261028|            1.0|               NaN|
|         9355|            1.0|               NaN|
|        10960|            1.0|               NaN|
|       139806|            0.0|               NaN|
|        20349|            1.0|               NaN|
|       261624|            1.0|               NaN|
|       259859|            0.0|               NaN|
|       260115|            0.0|               NaN|
|        48706|            0.0|

+----------+-------------+---------------+-----------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|f_user_vs_mean_purchase_from_LEAF_cat_in_last_90_days|
+----------+-------------+---------------+-----------------------------------------------------+
|  67114836|       261044|     2024-05-01|                                                 -1.0|
|  53132737|       261999|     2024-05-01|                                                 -1.0|
| 238279677|       117400|     2024-05-01|                                                 -1.0|
| 782596511|        52365|     2024-05-01|                                                 -1.0|
|2592723567|        52365|     2024-05-01|                                                 -1.0|
|1122553165|       178893|     2024-05-01|                                                 -1.0|
| 102058654|       178893|     2024-05-01|                                                 -1.0|
|  80471728|        74925|    

+----------+-------------+---------------+-------------------+
|  BUYER_ID|LEAF_CATEG_ID|EVENT_TIMESTAMP|user_purchase_count|
+----------+-------------+---------------+-------------------+
|    839622|        20696|     2024-05-01|                  2|
|   8248926|        23782|     2024-05-01|                  0|
|  11245860|        47173|     2024-05-01|                  2|
|  22447157|       261186|     2024-05-01|                  1|
| 102058654|       178893|     2024-05-01|                  0|
| 147993967|        29223|     2024-05-01|                  0|
| 209859385|       105516|     2024-05-01|                  0|
| 238279677|       117400|     2024-05-01|                  1|
| 326936611|          206|     2024-05-01|                  1|
| 556681734|        15053|     2024-05-01|                  0|
| 993558660|        20349|     2024-05-01|                  1|
|1033959804|       261186|     2024-05-01|                  1|
|1036833091|       121848|     2024-05-01|             

...

+----------+-------------+---------------+------------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|f_user_vs_mean_purchase_from_LVL2_cat_in_last_180_days|
+----------+-------------+---------------+------------------------------------------------------+
|  67852790|        42909|     2024-05-01|                                                  -1.0|
|1108288397|       183978|     2024-05-01|                                                  -1.0|
|1122553165|       178893|     2024-05-01|                                    0.7071067811865475|
| 102058654|       178893|     2024-05-01|                                   -0.7071067811865475|
|1854227088|        11897|     2024-05-01|                                                  -1.0|
| 895988726|         3197|     2024-05-01|                                                  -1.0|
|2537748086|        11863|     2024-05-01|                                                  -1.0|
|2541367954|        

+----------+-------------+---------------+-------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|user_purchase_count|
+----------+-------------+---------------+-------------------+
|2203671848|        11700|     2024-05-01|                  1|
| 124439030|        11700|     2024-05-01|                  0|
|1719030591|        64482|     2024-05-01|                  0|
|1173967650|        11233|     2024-05-01|                  0|
|1796827116|        11232|     2024-05-01|                  1|
|1948199593|        26395|     2024-05-01|                  0|
|1749424220|        11232|     2024-05-01|                  0|
|1444589293|        26395|     2024-05-01|                  0|
|1018245484|        26395|     2024-05-01|                  1|
|2541367954|        12576|     2024-05-01|                  0|
|  15191497|        11450|     2024-05-01|                  0|
|  18698040|        11116|     2024-05-01|                  1|
|1132069035|        64482|     2024-05-01|             

+-------------+-------------------+-------------------+
|META_CATEG_ID|    population_mean|  population_stddev|
+-------------+-------------------+-------------------+
|        11700| 2.8333333333333335|  5.238432634919365|
|        14339|                0.0|                NaN|
|          281|               0.25|                0.5|
|        11232| 0.3333333333333333| 0.5773502691896257|
|          293| 1.3333333333333333|  1.505545305418162|
|        11450|                0.0|                0.0|
|        26395|0.18181818181818182|0.40451991747794525|
|          625|                0.0|                NaN|
|        64482| 0.3333333333333333|  0.816496580927726|
|          260|               46.0|                NaN|
|        11116|                1.0|                NaN|
|        58058|                0.0|                NaN|
|            1|                0.5|  1.224744871391589|
|          220|                0.5| 0.7071067811865476|
|          267| 0.5714285714285714| 0.7867957924

+----------+-------------+---------------+-----------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|f_user_vs_mean_purchase_from_META_cat_in_last_30_days|
+----------+-------------+---------------+-----------------------------------------------------+
|1848882824|        11700|     2024-05-01|                                  -0.5408742520513384|
|  69144217|        11700|     2024-05-01|                                   1.9407840808900967|
| 182299961|        11700|     2024-05-01|                                  -0.5408742520513384|
|  62447387|        11700|     2024-05-01|                                  -0.5408742520513384|
|2203671848|        11700|     2024-05-01|                                  -0.3499774572096896|
|  17515348|        11700|     2024-05-01|                                   2.5134744654150434|
| 895988726|        11700|     2024-05-01|                                  -0.5408742520513384|
|    839622|        11700|    

+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+------------------------------------------------------+------------------------------------------------------+-----------------------------------------------------+
|  BUYER_ID|META_CATEG_ID|EVENT_TIMESTAMP|CATEG_LVL2_ID|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|     META_CATEG_NAME|f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days|f_user_vs_mean_purchase_from_LVL2_cat_in_last_180_days|f_user_vs_mean_purchase_from_META_cat_in_last_30_days|
+----------+-------------+---------------+-------------+-------------+---------------+-----+--------------------+--------------------+--------------------+------------------------------------------------------+------------------------------------------------------+-----------------------------------------------------+
|  11245860|          260|     2024-05-0

In [16]:
df_res_t.select( ["BUYER_ID", "LEAF_CATEG_ID","f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days"]).show()

+----------+-------------+------------------------------------------------------+
|  BUYER_ID|LEAF_CATEG_ID|f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days|
+----------+-------------+------------------------------------------------------+
|  11245860|        47173|                                                  -1.0|
|  11245860|       127413|                                                  -1.0|
| 168153910|       168058|                                                  -1.0|
|2541367954|        61754|                                                  -1.0|
|1784972362|       175815|                                                  -1.0|
|1159133810|       155183|                                                  -1.0|
|  62447387|       260923|                                                  -1.0|
|  18698040|       139806|                                                  -1.0|
|  82168277|        45246|                                                  -1.0|
|1796827116|    

In [5]:
import features as F

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF",  days=30,action_type="purchase")
df_res = F.add_feature(df_smp_sample, df_ft, join_key="LEAF_CATEG_ID", how="left") 

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF",  days=60,action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left") 

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF",  days=90,action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left") 

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF",  days=180,action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")  

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LEAF",  days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left") 

# Adding features for LVL2 category type
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp_sample, df_prch_sample, cat_type="LVL2", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

In [6]:
df_res.show()

+----------+-------------+---------------+-------------+---------------+-----+--------------------+--------------------+-------------+--------------------+-----------------------------------------------------+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------------+------------------------------------------------------+-----------------------------------------------------+
|  BUYER_ID|CATEG_LVL2_ID|EVENT_TIMESTAMP|LEAF_CATEG_ID|num_repurchases|label|     LEAF_CATEG_NAME|     CATEG_LVL2_NAME|META_CATEG_ID|     META_CATEG_NAME|f_user_vs_mean_purchase_from_LEAF_cat_in_last_30_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_60_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_90_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_180_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days|f_user_vs_mean_purchase_from_LVL2_cat_in_last_30_days|
+----------+-------------+----------

In [7]:
df_res.select("f_user_vs_mean_purchase_from_LEAF_cat_in_last_30_days",
               "f_user_vs_mean_purchase_from_LEAF_cat_in_last_60_days","f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days").show()

+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------------+
|f_user_vs_mean_purchase_from_LEAF_cat_in_last_30_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_60_days|f_user_vs_mean_purchase_from_LEAF_cat_in_last_360_days|
+-----------------------------------------------------+-----------------------------------------------------+------------------------------------------------------+
|                                                 -1.0|                                                 -1.0|                                                  -1.0|
|                                                 -1.0|                                                 -1.0|                                                  -1.0|
|                                                 -1.0|                                                 -1.0|                                                  -1.0|
|         

## Features comb

In [ ]:
#mean/max/min time- time -- number of days between the first view after a purchase:
df_ft = F.f_time_since_first_view_after_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LEAF")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_time_since_first_view_after_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LVL2")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_time_since_first_view_after_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="META")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

In [ ]:
#mean/max/min num_views_between_purchases:
df_ft = F.f_num_views_from_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LEAF")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID")

df_ft = F.f_num_views_from_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LVL2")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_num_views_from_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="META")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

In [ ]:
#Per user and category: num views since last purchase & Per user and category: time in days since last purchase 
df_ft = F.f_views_since_last_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LEAF")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID") 

df_ft = F.f_views_since_last_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="LVL2")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_views_since_last_purchase_in_cat(df_smp, df_prch, df_vi, cat_type="META")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

In [ ]:
#Also calc num views divided by the period of days from purchase to last view

In [ ]:
# category_diversity_entropy - diversity of purchases/views: 
df_ft = F.f_category_entropy(df_smp, df_prch, action_type="purchase", cat_type="LEAF")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left") 

df_ft = F.f_category_entropy(df_smp, df_prch,action_type="purchase", cat_type="LVL2")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_category_entropy(df_smp, df_prch,action_type="purchase", cat_type="META")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")

df_ft = F.f_category_entropy(df_smp, df_vi, action_type="view", cat_type="LEAF")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left") 

df_ft = F.f_category_entropy(df_smp, df_vi,action_type="view", cat_type="LVL2")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID")

df_ft = F.f_category_entropy(df_smp, df_vi,action_type="view", cat_type="META")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID")


In [ ]:
# Difference between mean purchases/views in period to user's own number of purchases/views in period so far

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LEAF", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

# Adding features for LVL2 category type
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="LVL2", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

# Adding features for META category type
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="META", days=30, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="META", days=60, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="META", days=90, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="META", days=180, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_prch, cat_type="META", days=360, action_type="purchase")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

# Adding features for LEAF category type for view action
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LEAF", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="LEAF_CATEG_ID", how="left")

# Adding features for LVL2 category type for view action
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="LVL2", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="CATEG_LVL2_ID", how="left")

# Adding features for META category type for view action
df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="META", days=30, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="META", days=60, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="META", days=90, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="META", days=180, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")

df_ft = F.f_user_vs_mean_cat_actions_in_last_x_days(df_smp, df_vi, cat_type="META", days=360, action_type="view")
df_res = F.add_feature(df_res, df_ft, join_key="META_CATEG_ID", how="left")


In [8]:
# Check the number of columns
num_columns = len(df_res.columns)
print(f"Number of columns: {num_columns}")

# Check the number of rows
num_rows = df_res.count()
print(f"Number of rows: {num_rows}")

Number of columns: 1243


Number of rows: 99999


In [9]:
save_table(session, df_res, "bx_ps_features_srch_1")

In [10]:
# create a smaller version only with key, features and label #("bx_ps_features_1") 
df = session.table("bx_ps_features_srch_1")
f_columns = [col for col in df.columns if col.startswith('f_')]
f_columns.extend(["BUYER_ID", "LEAF_CATEG_ID", "EVENT_TIMESTAMP", "label", "num_repurchases"])
df = df.select(f_columns)

# split data to train/validation/test sets
train, validation, test = df.randomSplit([0.7, 0.1, 0.2])

# save # prev bx_ps_features_train_1 , bx_ps_features_valid_1, bx_ps_features_test_1 
save_table(session, train, "bx_ps_features_train_srch_1")
save_table(session, validation, "bx_ps_features_valid_srch_1")
save_table(session, test, "bx_ps_features_test_srch_1")

In [ ]:
if session:
    session.stop()